<a href="https://colab.research.google.com/github/chawlaKat/Notebook-Noodlings/blob/master/Get_Scattered_Fancy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[insert intro here]

get_scattered got too long

im trying to make a decent-looking interactive scatter plot from .arff data

In [1]:
pip install liac_arff

  Stored in directory: /root/.cache/pip/wheels/d1/6a/e7/529dc54d76ecede4346164a09ae3168df358945612710f5203
Successfully built liac-arff


In [0]:
import bark_to_byte as bb
import altair as alt
import pandas as pd
from pprint import pprint

In [0]:
corgis = pd.read_csv('corgi_named.csv')

In [0]:
algSource = bb.arff_to_dataframe('abstracted_algorithm_runs.arff')

In [0]:
algMulti = algSource.set_index(['instance_id', 'algorithm'])
algUnstacked = algMulti.unstack()

flat_headers = []
for entry in algUnstacked.columns.values:
  flat_headers.append('_'.join(entry))
  
algFlat = algUnstacked.copy()
algFlat.columns = flat_headers

In [0]:
alg_rekeyed = algSource.set_index('instance_id')
alg_joined = alg_rekeyed.join(alg_rekeyed, lsuffix='_x', rsuffix='_y')
alg_joined.reset_index(inplace=True)

In [0]:
four_colors = ['#ffe200', '#f5ffff', '#6600db', '#11182b']
three_colors = ['#dd1f75', '#fda900', '#2a4dac']
three_colors_alt = ['#6754ad', '#ffffff', '#2d5d00']

In [0]:
def make_graph(data_source, x_axis, y_axis, point_marker = 'circle'):
  
  if(point_marker == 'circle'):
      new_graph = alt.Chart(data_source).mark_circle(size=60).encode(
          x = x_axis,
          y = y_axis
      )
      
  elif(point_marker == 'rect'):
      new_graph = alt.Chart(data_source).mark_rect().encode(
          x = x_axis,
          y = y_axis
      )
      
  elif(point_marker == 'bar'):
      new_graph = alt.Chart(data_source).mark_bar().encode(
          x = x_axis,
          y = y_axis
      )
    
  else:
      new_graph = alt.Chart(data_source).mark_point().encode(
          x = x_axis,
          y = y_axis
      )
  
  return new_graph

In [0]:
def make_dropdown(data_source, column, name_prefix):
  opts = list(data_source[column].unique())
  dropdown = alt.binding_select(options = opts)
  select = alt.selection_single(fields = [column],
                                bind = dropdown,
                                name = name_prefix
                               )
  
  return select

In [0]:
def make_selection(array_of_fields, is_multi = False):
  if (is_multi):
    selection = alt.selection_multi(fields = array_of_fields)
  else:
    selection = alt.selection_single(fields = array_of_fields)
    
  return selection

In [0]:
def make_color_scheme(data_source, column, colors, data_type_encoding = 'N'):
  scheme_domain = list(data_source[column].unique())
  scheme_range = colors
  
  scheme_scale = alt.Scale(domain = scheme_domain,
                    range = scheme_range)
  
  #altair needs to know if data is nominal, quantitative, ordinal, or temporal
  if(data_type_encoding == 'N' or data_type_encoding == 'Q' or
      data_type_encoding == 'O' or data_type_encoding == 'T'):
    column_typed = column + ':' + data_type_encoding
  else:
    column_typed = column + ':N'
  
  color_scheme = alt.Color(column_typed,
                           legend = None,
                           scale = scheme_scale)
  
  return color_scheme

In [0]:
def make_color_condition(selector, color_true, color_false = alt.value('darkgray')):
  condition = alt.condition(selector,
                            color_true,
                            color_false)
  
  return condition

In [0]:
def apply_selection_color(chart, selector, color_condition):
  colored_chart = chart.add_selection(
        selector
      ).encode(
        color = color_condition
      )
  
  return colored_chart

In [0]:
def create_comparative_scatter(data_source, key, x_category, y_category, 
                               x_data, y_data, x_filter, y_filter,
                               data_color):
  
  #filter data on x and y axis
  data_legend_base = make_graph(data_source, x_category, y_category, 'rect')
  data_legend_selector = make_selection([x_category, y_category])
  data_legend_colors = make_color_scheme(data_source, y_category, data_color)
  data_legend_update = make_color_condition(data_legend_selector, 
                                            data_legend_colors)
  data_legend = apply_selection_color(data_legend_base, data_legend_selector,
                                      data_legend_update)
  
  #filter data based on attribute
  filter_legend_base = make_graph(data_source, x_filter, y_filter, 'rect')
  filter_legend_selector = make_selection([x_filter, y_filter], True)
  filter_legend_colors = make_color_scheme(data_source, y_filter, data_color)
  filter_legend_update = make_color_condition(filter_legend_selector,
                                              filter_legend_colors)
  filter_legend = apply_selection_color(filter_legend_base, 
                                        filter_legend_selector, 
                                        filter_legend_update)
  
  #set up scatter plot to listen to both legends
  scatter_base = make_graph(data_source, x_data, y_data)
  scatter_tooltip = scatter_base.encode(
                                tooltip = [key, x_category, x_data, x_filter,
                                           y_category, y_data, y_filter]
                            )
  scatter_plot = scatter_tooltip.transform_filter(data_legend_selector
                            ).transform_filter(filter_legend_selector
                            ).interactive()
  
  
  comparative_chart = (data_legend & filter_legend) | scatter_plot
  
  return comparative_chart

In [17]:
algChart = make_graph(algSource, 'algorithm', 'sum(runtime)', 'bar')

algChart

Chart({
  data:    instance_id  repetition       algorithm   runtime runstatus
  0   Instance_0         1.0     algorithm_0  3600.000    memout
  1   Instance_0         1.0     algorithm_1  3600.000    memout
  2   Instance_0         1.0  id_algorithm_0   311.863        ok
  3   Instance_0         1.0  id_algorithm_1  3600.000   timeout
  4   Instance_1         1.0     algorithm_0  3600.000    memout
  5   Instance_1         1.0     algorithm_1  3600.000    memout
  6   Instance_1         1.0  id_algorithm_0    78.121        ok
  7   Instance_1         1.0  id_algorithm_1   600.402        ok
  8   Instance_2         1.0     algorithm_0  3600.000    memout
  9   Instance_2         1.0     algorithm_1  3600.000    memout
  10  Instance_2         1.0  id_algorithm_0     0.012        ok
  11  Instance_2         1.0  id_algorithm_1    60.720        ok
  12  Instance_3         1.0     algorithm_0  3600.000    memout
  13  Instance_3         1.0     algorithm_1    92.986        ok
  14  Instance_3         1.0  id_algorithm_0  2031.039        ok
  15  Instance_3         1.0  id_algorithm_1   141.857        ok
  16  Instance_4         1.0     algorithm_0  3600.000    memout
  17  Instance_4         1.0     algorithm_1  3600.000    memout
  18  Instance_4         1.0  id_algorithm_0   422.530        ok
  19  Instance_4         1.0  id_algorithm_1  3600.000   timeout
  20  Instance_5         1.0     algorithm_0   182.019        ok
  21  Instance_5         1.0     algorithm_1  3600.000    memout
  22  Instance_5         1.0  id_algorithm_0    66.988        ok
  23  Instance_5         1.0  id_algorithm_1   249.488        ok,
  encoding: FacetedEncoding({
    x: X({
      field: 'algorithm',
      type: 'nominal'
    }),
    y: Y({
      aggregate: 'sum',
      field: 'runtime',
      type: 'quantitative'
    })
  }),
  mark: 'bar'
})

In [18]:
instance_dropdown = make_dropdown(algSource, 'instance_id', 'in_id')

algDropdown = algChart.add_selection(
    instance_dropdown).transform_filter(
    instance_dropdown)

algDropdown

Chart({
  data:    instance_id  repetition       algorithm   runtime runstatus
  0   Instance_0         1.0     algorithm_0  3600.000    memout
  1   Instance_0         1.0     algorithm_1  3600.000    memout
  2   Instance_0         1.0  id_algorithm_0   311.863        ok
  3   Instance_0         1.0  id_algorithm_1  3600.000   timeout
  4   Instance_1         1.0     algorithm_0  3600.000    memout
  5   Instance_1         1.0     algorithm_1  3600.000    memout
  6   Instance_1         1.0  id_algorithm_0    78.121        ok
  7   Instance_1         1.0  id_algorithm_1   600.402        ok
  8   Instance_2         1.0     algorithm_0  3600.000    memout
  9   Instance_2         1.0     algorithm_1  3600.000    memout
  10  Instance_2         1.0  id_algorithm_0     0.012        ok
  11  Instance_2         1.0  id_algorithm_1    60.720        ok
  12  Instance_3         1.0     algorithm_0  3600.000    memout
  13  Instance_3         1.0     algorithm_1    92.986        ok
  14  Instance_3         1.0  id_algorithm_0  2031.039        ok
  15  Instance_3         1.0  id_algorithm_1   141.857        ok
  16  Instance_4         1.0     algorithm_0  3600.000    memout
  17  Instance_4         1.0     algorithm_1  3600.000    memout
  18  Instance_4         1.0  id_algorithm_0   422.530        ok
  19  Instance_4         1.0  id_algorithm_1  3600.000   timeout
  20  Instance_5         1.0     algorithm_0   182.019        ok
  21  Instance_5         1.0     algorithm_1  3600.000    memout
  22  Instance_5         1.0  id_algorithm_0    66.988        ok
  23  Instance_5         1.0  id_algorithm_1   249.488        ok,
  encoding: FacetedEncoding({
    x: X({
      field: 'algorithm',
      type: 'nominal'
    }),
    y: Y({
      aggregate: 'sum',
      field: 'runtime',
      type: 'quantitative'
    })
  }),
  mark: 'bar',
  selection: {'in_id': SelectionDef({
    bind: BindRadioSelect({
      input: 'select',
      options: ['Instance_0', 'Instance_1', 'Instance_2', 'Instance_3', 'Instance_4', 'Instance_5']
    }),
    fields: ['instance_id'],
    type: 'single'
  })},
  transform: [FilterTransform({
    filter: {'selection': 'in_id'}
  })]
})

In [19]:
alg_legend = make_graph(alg_joined, 'algorithm_x', 'algorithm_y', 'rect')

alg_legend

Chart({
  data:    instance_id  repetition_x  ... runtime_y  runstatus_y
  0   Instance_0           1.0  ...  3600.000       memout
  1   Instance_0           1.0  ...  3600.000       memout
  2   Instance_0           1.0  ...   311.863           ok
  3   Instance_0           1.0  ...  3600.000      timeout
  4   Instance_0           1.0  ...  3600.000       memout
  5   Instance_0           1.0  ...  3600.000       memout
  6   Instance_0           1.0  ...   311.863           ok
  7   Instance_0           1.0  ...  3600.000      timeout
  8   Instance_0           1.0  ...  3600.000       memout
  9   Instance_0           1.0  ...  3600.000       memout
  10  Instance_0           1.0  ...   311.863           ok
  11  Instance_0           1.0  ...  3600.000      timeout
  12  Instance_0           1.0  ...  3600.000       memout
  13  Instance_0           1.0  ...  3600.000       memout
  14  Instance_0           1.0  ...   311.863           ok
  15  Instance_0           1.0  ...  3600.000      timeout
  16  Instance_1           1.0  ...  3600.000       memout
  17  Instance_1           1.0  ...  3600.000       memout
  18  Instance_1           1.0  ...    78.121           ok
  19  Instance_1           1.0  ...   600.402           ok
  20  Instance_1           1.0  ...  3600.000       memout
  21  Instance_1           1.0  ...  3600.000       memout
  22  Instance_1           1.0  ...    78.121           ok
  23  Instance_1           1.0  ...   600.402           ok
  24  Instance_1           1.0  ...  3600.000       memout
  25  Instance_1           1.0  ...  3600.000       memout
  26  Instance_1           1.0  ...    78.121           ok
  27  Instance_1           1.0  ...   600.402           ok
  28  Instance_1           1.0  ...  3600.000       memout
  29  Instance_1           1.0  ...  3600.000       memout
  ..         ...           ...  ...       ...          ...
  66  Instance_4           1.0  ...   422.530           ok
  67  Instance_4           1.0  ...  3600.000      timeout
  68  Instance_4           1.0  ...  3600.000       memout
  69  Instance_4           1.0  ...  3600.000       memout
  70  Instance_4           1.0  ...   422.530           ok
  71  Instance_4           1.0  ...  3600.000      timeout
  72  Instance_4           1.0  ...  3600.000       memout
  73  Instance_4           1.0  ...  3600.000       memout
  74  Instance_4           1.0  ...   422.530           ok
  75  Instance_4           1.0  ...  3600.000      timeout
  76  Instance_4           1.0  ...  3600.000       memout
  77  Instance_4           1.0  ...  3600.000       memout
  78  Instance_4           1.0  ...   422.530           ok
  79  Instance_4           1.0  ...  3600.000      timeout
  80  Instance_5           1.0  ...   182.019           ok
  81  Instance_5           1.0  ...  3600.000       memout
  82  Instance_5           1.0  ...    66.988           ok
  83  Instance_5           1.0  ...   249.488           ok
  84  Instance_5           1.0  ...   182.019           ok
  85  Instance_5           1.0  ...  3600.000       memout
  86  Instance_5           1.0  ...    66.988           ok
  87  Instance_5           1.0  ...   249.488           ok
  88  Instance_5           1.0  ...   182.019           ok
  89  Instance_5           1.0  ...  3600.000       memout
  90  Instance_5           1.0  ...    66.988           ok
  91  Instance_5           1.0  ...   249.488           ok
  92  Instance_5           1.0  ...   182.019           ok
  93  Instance_5           1.0  ...  3600.000       memout
  94  Instance_5           1.0  ...    66.988           ok
  95  Instance_5           1.0  ...   249.488           ok
  
  [96 rows x 9 columns],
  encoding: FacetedEncoding({
    x: X({
      field: 'algorithm_x',
      type: 'nominal'
    }),
    y: Y({
      field: 'algorithm_y',
      type: 'nominal'
    })
  }),
  mark: 'rect'
})

In [20]:
alg_colors = make_color_scheme(alg_joined, 'algorithm_y', four_colors)

alg_legend_colored = alg_legend.encode(
    color = alg_colors
)

alg_legend_colored

Chart({
  data:    instance_id  repetition_x  ... runtime_y  runstatus_y
  0   Instance_0           1.0  ...  3600.000       memout
  1   Instance_0           1.0  ...  3600.000       memout
  2   Instance_0           1.0  ...   311.863           ok
  3   Instance_0           1.0  ...  3600.000      timeout
  4   Instance_0           1.0  ...  3600.000       memout
  5   Instance_0           1.0  ...  3600.000       memout
  6   Instance_0           1.0  ...   311.863           ok
  7   Instance_0           1.0  ...  3600.000      timeout
  8   Instance_0           1.0  ...  3600.000       memout
  9   Instance_0           1.0  ...  3600.000       memout
  10  Instance_0           1.0  ...   311.863           ok
  11  Instance_0           1.0  ...  3600.000      timeout
  12  Instance_0           1.0  ...  3600.000       memout
  13  Instance_0           1.0  ...  3600.000       memout
  14  Instance_0           1.0  ...   311.863           ok
  15  Instance_0           1.0  ...  3600.000      timeout
  16  Instance_1           1.0  ...  3600.000       memout
  17  Instance_1           1.0  ...  3600.000       memout
  18  Instance_1           1.0  ...    78.121           ok
  19  Instance_1           1.0  ...   600.402           ok
  20  Instance_1           1.0  ...  3600.000       memout
  21  Instance_1           1.0  ...  3600.000       memout
  22  Instance_1           1.0  ...    78.121           ok
  23  Instance_1           1.0  ...   600.402           ok
  24  Instance_1           1.0  ...  3600.000       memout
  25  Instance_1           1.0  ...  3600.000       memout
  26  Instance_1           1.0  ...    78.121           ok
  27  Instance_1           1.0  ...   600.402           ok
  28  Instance_1           1.0  ...  3600.000       memout
  29  Instance_1           1.0  ...  3600.000       memout
  ..         ...           ...  ...       ...          ...
  66  Instance_4           1.0  ...   422.530           ok
  67  Instance_4           1.0  ...  3600.000      timeout
  68  Instance_4           1.0  ...  3600.000       memout
  69  Instance_4           1.0  ...  3600.000       memout
  70  Instance_4           1.0  ...   422.530           ok
  71  Instance_4           1.0  ...  3600.000      timeout
  72  Instance_4           1.0  ...  3600.000       memout
  73  Instance_4           1.0  ...  3600.000       memout
  74  Instance_4           1.0  ...   422.530           ok
  75  Instance_4           1.0  ...  3600.000      timeout
  76  Instance_4           1.0  ...  3600.000       memout
  77  Instance_4           1.0  ...  3600.000       memout
  78  Instance_4           1.0  ...   422.530           ok
  79  Instance_4           1.0  ...  3600.000      timeout
  80  Instance_5           1.0  ...   182.019           ok
  81  Instance_5           1.0  ...  3600.000       memout
  82  Instance_5           1.0  ...    66.988           ok
  83  Instance_5           1.0  ...   249.488           ok
  84  Instance_5           1.0  ...   182.019           ok
  85  Instance_5           1.0  ...  3600.000       memout
  86  Instance_5           1.0  ...    66.988           ok
  87  Instance_5           1.0  ...   249.488           ok
  88  Instance_5           1.0  ...   182.019           ok
  89  Instance_5           1.0  ...  3600.000       memout
  90  Instance_5           1.0  ...    66.988           ok
  91  Instance_5           1.0  ...   249.488           ok
  92  Instance_5           1.0  ...   182.019           ok
  93  Instance_5           1.0  ...  3600.000       memout
  94  Instance_5           1.0  ...    66.988           ok
  95  Instance_5           1.0  ...   249.488           ok
  
  [96 rows x 9 columns],
  encoding: FacetedEncoding({
    color: Color({
      field: 'algorithm_y',
      legend: None,
      scale: Scale({
        domain: ['algorithm_0', 'algorithm_1', 'id_algorithm_0', 'id_algorithm_1'],
        range: ['#ffe200', '#f5ffff', '#6600db', '#11182b']
      }),
      type: 'nominal'
    })

In [21]:
alg_color_scheme = make_color_scheme(algSource, 'algorithm', four_colors)

alg_chart_colored = algChart.encode(color = alg_color_scheme)

alg_chart_colored

Chart({
  data:    instance_id  repetition       algorithm   runtime runstatus
  0   Instance_0         1.0     algorithm_0  3600.000    memout
  1   Instance_0         1.0     algorithm_1  3600.000    memout
  2   Instance_0         1.0  id_algorithm_0   311.863        ok
  3   Instance_0         1.0  id_algorithm_1  3600.000   timeout
  4   Instance_1         1.0     algorithm_0  3600.000    memout
  5   Instance_1         1.0     algorithm_1  3600.000    memout
  6   Instance_1         1.0  id_algorithm_0    78.121        ok
  7   Instance_1         1.0  id_algorithm_1   600.402        ok
  8   Instance_2         1.0     algorithm_0  3600.000    memout
  9   Instance_2         1.0     algorithm_1  3600.000    memout
  10  Instance_2         1.0  id_algorithm_0     0.012        ok
  11  Instance_2         1.0  id_algorithm_1    60.720        ok
  12  Instance_3         1.0     algorithm_0  3600.000    memout
  13  Instance_3         1.0     algorithm_1    92.986        ok
  14  Instance_3         1.0  id_algorithm_0  2031.039        ok
  15  Instance_3         1.0  id_algorithm_1   141.857        ok
  16  Instance_4         1.0     algorithm_0  3600.000    memout
  17  Instance_4         1.0     algorithm_1  3600.000    memout
  18  Instance_4         1.0  id_algorithm_0   422.530        ok
  19  Instance_4         1.0  id_algorithm_1  3600.000   timeout
  20  Instance_5         1.0     algorithm_0   182.019        ok
  21  Instance_5         1.0     algorithm_1  3600.000    memout
  22  Instance_5         1.0  id_algorithm_0    66.988        ok
  23  Instance_5         1.0  id_algorithm_1   249.488        ok,
  encoding: FacetedEncoding({
    color: Color({
      field: 'algorithm',
      legend: None,
      scale: Scale({
        domain: ['algorithm_0', 'algorithm_1', 'id_algorithm_0', 'id_algorithm_1'],
        range: ['#ffe200', '#f5ffff', '#6600db', '#11182b']
      }),
      type: 'nominal'
    }),
    x: X({
      field: 'algorithm',
      type: 'nominal'
    }),
    y: Y({
      aggregate: 'sum',
      field: 'runtime',
      type: 'quantitative'
    })
  }),
  mark: 'bar'
})

In [22]:
alg_selection = make_selection(['algorithm'])

with_select = algChart.add_selection(alg_selection).transform_filter(alg_selection)
with_select

Chart({
  data:    instance_id  repetition       algorithm   runtime runstatus
  0   Instance_0         1.0     algorithm_0  3600.000    memout
  1   Instance_0         1.0     algorithm_1  3600.000    memout
  2   Instance_0         1.0  id_algorithm_0   311.863        ok
  3   Instance_0         1.0  id_algorithm_1  3600.000   timeout
  4   Instance_1         1.0     algorithm_0  3600.000    memout
  5   Instance_1         1.0     algorithm_1  3600.000    memout
  6   Instance_1         1.0  id_algorithm_0    78.121        ok
  7   Instance_1         1.0  id_algorithm_1   600.402        ok
  8   Instance_2         1.0     algorithm_0  3600.000    memout
  9   Instance_2         1.0     algorithm_1  3600.000    memout
  10  Instance_2         1.0  id_algorithm_0     0.012        ok
  11  Instance_2         1.0  id_algorithm_1    60.720        ok
  12  Instance_3         1.0     algorithm_0  3600.000    memout
  13  Instance_3         1.0     algorithm_1    92.986        ok
  14  Instance_3         1.0  id_algorithm_0  2031.039        ok
  15  Instance_3         1.0  id_algorithm_1   141.857        ok
  16  Instance_4         1.0     algorithm_0  3600.000    memout
  17  Instance_4         1.0     algorithm_1  3600.000    memout
  18  Instance_4         1.0  id_algorithm_0   422.530        ok
  19  Instance_4         1.0  id_algorithm_1  3600.000   timeout
  20  Instance_5         1.0     algorithm_0   182.019        ok
  21  Instance_5         1.0     algorithm_1  3600.000    memout
  22  Instance_5         1.0  id_algorithm_0    66.988        ok
  23  Instance_5         1.0  id_algorithm_1   249.488        ok,
  encoding: FacetedEncoding({
    x: X({
      field: 'algorithm',
      type: 'nominal'
    }),
    y: Y({
      aggregate: 'sum',
      field: 'runtime',
      type: 'quantitative'
    })
  }),
  mark: 'bar',
  selection: {'selector001': SelectionDef({
    fields: ['algorithm'],
    type: 'single'
  })},
  transform: [FilterTransform({
    filter: {'selection': 'selector001'}
  })]
})

In [23]:
alg_selector = make_selection(['algorithm'])
alg_color = make_color_scheme(algSource, 'algorithm', four_colors)
alg_color_update = make_color_condition(alg_selector, alg_color)

fresh_chart = algChart.add_selection(alg_selector).encode(color = alg_color_update)

fresh_chart

Chart({
  data:    instance_id  repetition       algorithm   runtime runstatus
  0   Instance_0         1.0     algorithm_0  3600.000    memout
  1   Instance_0         1.0     algorithm_1  3600.000    memout
  2   Instance_0         1.0  id_algorithm_0   311.863        ok
  3   Instance_0         1.0  id_algorithm_1  3600.000   timeout
  4   Instance_1         1.0     algorithm_0  3600.000    memout
  5   Instance_1         1.0     algorithm_1  3600.000    memout
  6   Instance_1         1.0  id_algorithm_0    78.121        ok
  7   Instance_1         1.0  id_algorithm_1   600.402        ok
  8   Instance_2         1.0     algorithm_0  3600.000    memout
  9   Instance_2         1.0     algorithm_1  3600.000    memout
  10  Instance_2         1.0  id_algorithm_0     0.012        ok
  11  Instance_2         1.0  id_algorithm_1    60.720        ok
  12  Instance_3         1.0     algorithm_0  3600.000    memout
  13  Instance_3         1.0     algorithm_1    92.986        ok
  14  Instance_3         1.0  id_algorithm_0  2031.039        ok
  15  Instance_3         1.0  id_algorithm_1   141.857        ok
  16  Instance_4         1.0     algorithm_0  3600.000    memout
  17  Instance_4         1.0     algorithm_1  3600.000    memout
  18  Instance_4         1.0  id_algorithm_0   422.530        ok
  19  Instance_4         1.0  id_algorithm_1  3600.000   timeout
  20  Instance_5         1.0     algorithm_0   182.019        ok
  21  Instance_5         1.0     algorithm_1  3600.000    memout
  22  Instance_5         1.0  id_algorithm_0    66.988        ok
  23  Instance_5         1.0  id_algorithm_1   249.488        ok,
  encoding: FacetedEncoding({
    color: ColorValue({
      condition: Color({
        field: FieldName('algorithm'),
        legend: None,
        scale: Scale({
          domain: ['algorithm_0', 'algorithm_1', 'id_algorithm_0', 'id_algorithm_1'],
          range: ['#ffe200', '#f5ffff', '#6600db', '#11182b']
        }),
        selection: 'selector002',
        type: StandardType('nominal')
      }),
      value: 'darkgray'
    }),
    x: X({
      field: 'algorithm',
      type: 'nominal'
    }),
    y: Y({
      aggregate: 'sum',
      field: 'runtime',
      type: 'quantitative'
    })
  }),
  mark: 'bar',
  selection: {'selector002': SelectionDef({
    fields: ['algorithm'],
    type: 'single'
  })}
})

In [24]:
colorable_chart = apply_selection_color(algChart, alg_selector, alg_color_update)
                                        
colorable_chart

Chart({
  data:    instance_id  repetition       algorithm   runtime runstatus
  0   Instance_0         1.0     algorithm_0  3600.000    memout
  1   Instance_0         1.0     algorithm_1  3600.000    memout
  2   Instance_0         1.0  id_algorithm_0   311.863        ok
  3   Instance_0         1.0  id_algorithm_1  3600.000   timeout
  4   Instance_1         1.0     algorithm_0  3600.000    memout
  5   Instance_1         1.0     algorithm_1  3600.000    memout
  6   Instance_1         1.0  id_algorithm_0    78.121        ok
  7   Instance_1         1.0  id_algorithm_1   600.402        ok
  8   Instance_2         1.0     algorithm_0  3600.000    memout
  9   Instance_2         1.0     algorithm_1  3600.000    memout
  10  Instance_2         1.0  id_algorithm_0     0.012        ok
  11  Instance_2         1.0  id_algorithm_1    60.720        ok
  12  Instance_3         1.0     algorithm_0  3600.000    memout
  13  Instance_3         1.0     algorithm_1    92.986        ok
  14  Instance_3         1.0  id_algorithm_0  2031.039        ok
  15  Instance_3         1.0  id_algorithm_1   141.857        ok
  16  Instance_4         1.0     algorithm_0  3600.000    memout
  17  Instance_4         1.0     algorithm_1  3600.000    memout
  18  Instance_4         1.0  id_algorithm_0   422.530        ok
  19  Instance_4         1.0  id_algorithm_1  3600.000   timeout
  20  Instance_5         1.0     algorithm_0   182.019        ok
  21  Instance_5         1.0     algorithm_1  3600.000    memout
  22  Instance_5         1.0  id_algorithm_0    66.988        ok
  23  Instance_5         1.0  id_algorithm_1   249.488        ok,
  encoding: FacetedEncoding({
    color: ColorValue({
      condition: Color({
        field: FieldName('algorithm'),
        legend: None,
        scale: Scale({
          domain: ['algorithm_0', 'algorithm_1', 'id_algorithm_0', 'id_algorithm_1'],
          range: ['#ffe200', '#f5ffff', '#6600db', '#11182b']
        }),
        selection: 'selector002',
        type: StandardType('nominal')
      }),
      value: 'darkgray'
    }),
    x: X({
      field: 'algorithm',
      type: 'nominal'
    }),
    y: Y({
      aggregate: 'sum',
      field: 'runtime',
      type: 'quantitative'
    })
  }),
  mark: 'bar',
  selection: {'selector002': SelectionDef({
    fields: ['algorithm'],
    type: 'single'
  })}
})

In [25]:
legend_base = make_graph(alg_joined, 'runstatus_x', 'runstatus_y', 'rect')
legend_select = make_selection(['runstatus_x', 'runstatus_y'], True)
legend_colors = make_color_scheme(alg_joined, 'runstatus_y', three_colors)
legend_color_update = make_color_condition(legend_select, legend_colors, alt.value('darkgray'))
legend = apply_selection_color(legend_base, legend_select, legend_color_update)

legend

Chart({
  data:    instance_id  repetition_x  ... runtime_y  runstatus_y
  0   Instance_0           1.0  ...  3600.000       memout
  1   Instance_0           1.0  ...  3600.000       memout
  2   Instance_0           1.0  ...   311.863           ok
  3   Instance_0           1.0  ...  3600.000      timeout
  4   Instance_0           1.0  ...  3600.000       memout
  5   Instance_0           1.0  ...  3600.000       memout
  6   Instance_0           1.0  ...   311.863           ok
  7   Instance_0           1.0  ...  3600.000      timeout
  8   Instance_0           1.0  ...  3600.000       memout
  9   Instance_0           1.0  ...  3600.000       memout
  10  Instance_0           1.0  ...   311.863           ok
  11  Instance_0           1.0  ...  3600.000      timeout
  12  Instance_0           1.0  ...  3600.000       memout
  13  Instance_0           1.0  ...  3600.000       memout
  14  Instance_0           1.0  ...   311.863           ok
  15  Instance_0           1.0  ...  3600.000      timeout
  16  Instance_1           1.0  ...  3600.000       memout
  17  Instance_1           1.0  ...  3600.000       memout
  18  Instance_1           1.0  ...    78.121           ok
  19  Instance_1           1.0  ...   600.402           ok
  20  Instance_1           1.0  ...  3600.000       memout
  21  Instance_1           1.0  ...  3600.000       memout
  22  Instance_1           1.0  ...    78.121           ok
  23  Instance_1           1.0  ...   600.402           ok
  24  Instance_1           1.0  ...  3600.000       memout
  25  Instance_1           1.0  ...  3600.000       memout
  26  Instance_1           1.0  ...    78.121           ok
  27  Instance_1           1.0  ...   600.402           ok
  28  Instance_1           1.0  ...  3600.000       memout
  29  Instance_1           1.0  ...  3600.000       memout
  ..         ...           ...  ...       ...          ...
  66  Instance_4           1.0  ...   422.530           ok
  67  Instance_4           1.0  ...  3600.000      timeout
  68  Instance_4           1.0  ...  3600.000       memout
  69  Instance_4           1.0  ...  3600.000       memout
  70  Instance_4           1.0  ...   422.530           ok
  71  Instance_4           1.0  ...  3600.000      timeout
  72  Instance_4           1.0  ...  3600.000       memout
  73  Instance_4           1.0  ...  3600.000       memout
  74  Instance_4           1.0  ...   422.530           ok
  75  Instance_4           1.0  ...  3600.000      timeout
  76  Instance_4           1.0  ...  3600.000       memout
  77  Instance_4           1.0  ...  3600.000       memout
  78  Instance_4           1.0  ...   422.530           ok
  79  Instance_4           1.0  ...  3600.000      timeout
  80  Instance_5           1.0  ...   182.019           ok
  81  Instance_5           1.0  ...  3600.000       memout
  82  Instance_5           1.0  ...    66.988           ok
  83  Instance_5           1.0  ...   249.488           ok
  84  Instance_5           1.0  ...   182.019           ok
  85  Instance_5           1.0  ...  3600.000       memout
  86  Instance_5           1.0  ...    66.988           ok
  87  Instance_5           1.0  ...   249.488           ok
  88  Instance_5           1.0  ...   182.019           ok
  89  Instance_5           1.0  ...  3600.000       memout
  90  Instance_5           1.0  ...    66.988           ok
  91  Instance_5           1.0  ...   249.488           ok
  92  Instance_5           1.0  ...   182.019           ok
  93  Instance_5           1.0  ...  3600.000       memout
  94  Instance_5           1.0  ...    66.988           ok
  95  Instance_5           1.0  ...   249.488           ok
  
  [96 rows x 9 columns],
  encoding: FacetedEncoding({
    color: ColorValue({
      condition: Color({
        field: FieldName('runstatus_y'),
        legend: None,
        scale: Scale({
          domain: ['memout', 'ok', 'timeout'],
          range: ['#dd1f75', '#fda900', '#2a4dac']
        }),
        selection: 'selec

In [26]:
a_leg_base = make_graph(alg_joined, 'algorithm_x', 'algorithm_y', 'rect')
a_leg_select = make_selection(['algorithm_x', 'algorithm_y'])
a_leg_colors = make_color_scheme(alg_joined, 'algorithm_y', four_colors)
a_leg_col_update = make_color_condition(a_leg_select, a_leg_colors)
alg_legend = apply_selection_color(a_leg_base, a_leg_select, a_leg_col_update)

alg_legend

Chart({
  data:    instance_id  repetition_x  ... runtime_y  runstatus_y
  0   Instance_0           1.0  ...  3600.000       memout
  1   Instance_0           1.0  ...  3600.000       memout
  2   Instance_0           1.0  ...   311.863           ok
  3   Instance_0           1.0  ...  3600.000      timeout
  4   Instance_0           1.0  ...  3600.000       memout
  5   Instance_0           1.0  ...  3600.000       memout
  6   Instance_0           1.0  ...   311.863           ok
  7   Instance_0           1.0  ...  3600.000      timeout
  8   Instance_0           1.0  ...  3600.000       memout
  9   Instance_0           1.0  ...  3600.000       memout
  10  Instance_0           1.0  ...   311.863           ok
  11  Instance_0           1.0  ...  3600.000      timeout
  12  Instance_0           1.0  ...  3600.000       memout
  13  Instance_0           1.0  ...  3600.000       memout
  14  Instance_0           1.0  ...   311.863           ok
  15  Instance_0           1.0  ...  3600.000      timeout
  16  Instance_1           1.0  ...  3600.000       memout
  17  Instance_1           1.0  ...  3600.000       memout
  18  Instance_1           1.0  ...    78.121           ok
  19  Instance_1           1.0  ...   600.402           ok
  20  Instance_1           1.0  ...  3600.000       memout
  21  Instance_1           1.0  ...  3600.000       memout
  22  Instance_1           1.0  ...    78.121           ok
  23  Instance_1           1.0  ...   600.402           ok
  24  Instance_1           1.0  ...  3600.000       memout
  25  Instance_1           1.0  ...  3600.000       memout
  26  Instance_1           1.0  ...    78.121           ok
  27  Instance_1           1.0  ...   600.402           ok
  28  Instance_1           1.0  ...  3600.000       memout
  29  Instance_1           1.0  ...  3600.000       memout
  ..         ...           ...  ...       ...          ...
  66  Instance_4           1.0  ...   422.530           ok
  67  Instance_4           1.0  ...  3600.000      timeout
  68  Instance_4           1.0  ...  3600.000       memout
  69  Instance_4           1.0  ...  3600.000       memout
  70  Instance_4           1.0  ...   422.530           ok
  71  Instance_4           1.0  ...  3600.000      timeout
  72  Instance_4           1.0  ...  3600.000       memout
  73  Instance_4           1.0  ...  3600.000       memout
  74  Instance_4           1.0  ...   422.530           ok
  75  Instance_4           1.0  ...  3600.000      timeout
  76  Instance_4           1.0  ...  3600.000       memout
  77  Instance_4           1.0  ...  3600.000       memout
  78  Instance_4           1.0  ...   422.530           ok
  79  Instance_4           1.0  ...  3600.000      timeout
  80  Instance_5           1.0  ...   182.019           ok
  81  Instance_5           1.0  ...  3600.000       memout
  82  Instance_5           1.0  ...    66.988           ok
  83  Instance_5           1.0  ...   249.488           ok
  84  Instance_5           1.0  ...   182.019           ok
  85  Instance_5           1.0  ...  3600.000       memout
  86  Instance_5           1.0  ...    66.988           ok
  87  Instance_5           1.0  ...   249.488           ok
  88  Instance_5           1.0  ...   182.019           ok
  89  Instance_5           1.0  ...  3600.000       memout
  90  Instance_5           1.0  ...    66.988           ok
  91  Instance_5           1.0  ...   249.488           ok
  92  Instance_5           1.0  ...   182.019           ok
  93  Instance_5           1.0  ...  3600.000       memout
  94  Instance_5           1.0  ...    66.988           ok
  95  Instance_5           1.0  ...   249.488           ok
  
  [96 rows x 9 columns],
  encoding: FacetedEncoding({
    color: ColorValue({
      condition: Color({
        field: FieldName('algorithm_y'),
        legend: None,
        scale: Scale({
          domain: ['algorithm_0', 'algorithm_1', 'id_algorithm_0', 'id_algorithm_1'],
          range: ['#ffe200', '#f5ffff', '#6600db'

In [27]:
scatter_base = make_graph(alg_joined, 'runtime_x', 'runtime_y')
scatter_filtered = scatter_base.transform_filter(
                      legend_select
                   ).transform_filter(
                      a_leg_select
                   ).interactive()

(alg_legend & legend) | scatter_filtered

HConcatChart({
  data:    instance_id  repetition_x  ... runtime_y  runstatus_y
  0   Instance_0           1.0  ...  3600.000       memout
  1   Instance_0           1.0  ...  3600.000       memout
  2   Instance_0           1.0  ...   311.863           ok
  3   Instance_0           1.0  ...  3600.000      timeout
  4   Instance_0           1.0  ...  3600.000       memout
  5   Instance_0           1.0  ...  3600.000       memout
  6   Instance_0           1.0  ...   311.863           ok
  7   Instance_0           1.0  ...  3600.000      timeout
  8   Instance_0           1.0  ...  3600.000       memout
  9   Instance_0           1.0  ...  3600.000       memout
  10  Instance_0           1.0  ...   311.863           ok
  11  Instance_0           1.0  ...  3600.000      timeout
  12  Instance_0           1.0  ...  3600.000       memout
  13  Instance_0           1.0  ...  3600.000       memout
  14  Instance_0           1.0  ...   311.863           ok
  15  Instance_0           1.0  ...  3600.000      timeout
  16  Instance_1           1.0  ...  3600.000       memout
  17  Instance_1           1.0  ...  3600.000       memout
  18  Instance_1           1.0  ...    78.121           ok
  19  Instance_1           1.0  ...   600.402           ok
  20  Instance_1           1.0  ...  3600.000       memout
  21  Instance_1           1.0  ...  3600.000       memout
  22  Instance_1           1.0  ...    78.121           ok
  23  Instance_1           1.0  ...   600.402           ok
  24  Instance_1           1.0  ...  3600.000       memout
  25  Instance_1           1.0  ...  3600.000       memout
  26  Instance_1           1.0  ...    78.121           ok
  27  Instance_1           1.0  ...   600.402           ok
  28  Instance_1           1.0  ...  3600.000       memout
  29  Instance_1           1.0  ...  3600.000       memout
  ..         ...           ...  ...       ...          ...
  66  Instance_4           1.0  ...   422.530           ok
  67  Instance_4           1.0  ...  3600.000      timeout
  68  Instance_4           1.0  ...  3600.000       memout
  69  Instance_4           1.0  ...  3600.000       memout
  70  Instance_4           1.0  ...   422.530           ok
  71  Instance_4           1.0  ...  3600.000      timeout
  72  Instance_4           1.0  ...  3600.000       memout
  73  Instance_4           1.0  ...  3600.000       memout
  74  Instance_4           1.0  ...   422.530           ok
  75  Instance_4           1.0  ...  3600.000      timeout
  76  Instance_4           1.0  ...  3600.000       memout
  77  Instance_4           1.0  ...  3600.000       memout
  78  Instance_4           1.0  ...   422.530           ok
  79  Instance_4           1.0  ...  3600.000      timeout
  80  Instance_5           1.0  ...   182.019           ok
  81  Instance_5           1.0  ...  3600.000       memout
  82  Instance_5           1.0  ...    66.988           ok
  83  Instance_5           1.0  ...   249.488           ok
  84  Instance_5           1.0  ...   182.019           ok
  85  Instance_5           1.0  ...  3600.000       memout
  86  Instance_5           1.0  ...    66.988           ok
  87  Instance_5           1.0  ...   249.488           ok
  88  Instance_5           1.0  ...   182.019           ok
  89  Instance_5           1.0  ...  3600.000       memout
  90  Instance_5           1.0  ...    66.988           ok
  91  Instance_5           1.0  ...   249.488           ok
  92  Instance_5           1.0  ...   182.019           ok
  93  Instance_5           1.0  ...  3600.000       memout
  94  Instance_5           1.0  ...    66.988           ok
  95  Instance_5           1.0  ...   249.488           ok
  
  [96 rows x 9 columns],
  hconcat: [VConcatChart({
    vconcat: [Chart({
      encoding: FacetedEncoding({
        color: ColorValue({
          condition: Color({
            field: FieldName('algorithm_y'),
            legend: None,
            scale: Scale({
              domain: ['algorithm_0', 'algorithm_1', 'i

In [28]:
create_comparative_scatter(alg_joined, 'instance_id', 'algorithm_x', 'algorithm_y',
                           'runtime_x', 'runtime_y', 'runstatus_x', 
                           'runstatus_y', four_colors)

HConcatChart({
  data:    instance_id  repetition_x  ... runtime_y  runstatus_y
  0   Instance_0           1.0  ...  3600.000       memout
  1   Instance_0           1.0  ...  3600.000       memout
  2   Instance_0           1.0  ...   311.863           ok
  3   Instance_0           1.0  ...  3600.000      timeout
  4   Instance_0           1.0  ...  3600.000       memout
  5   Instance_0           1.0  ...  3600.000       memout
  6   Instance_0           1.0  ...   311.863           ok
  7   Instance_0           1.0  ...  3600.000      timeout
  8   Instance_0           1.0  ...  3600.000       memout
  9   Instance_0           1.0  ...  3600.000       memout
  10  Instance_0           1.0  ...   311.863           ok
  11  Instance_0           1.0  ...  3600.000      timeout
  12  Instance_0           1.0  ...  3600.000       memout
  13  Instance_0           1.0  ...  3600.000       memout
  14  Instance_0           1.0  ...   311.863           ok
  15  Instance_0           1.0  ...  3600.000      timeout
  16  Instance_1           1.0  ...  3600.000       memout
  17  Instance_1           1.0  ...  3600.000       memout
  18  Instance_1           1.0  ...    78.121           ok
  19  Instance_1           1.0  ...   600.402           ok
  20  Instance_1           1.0  ...  3600.000       memout
  21  Instance_1           1.0  ...  3600.000       memout
  22  Instance_1           1.0  ...    78.121           ok
  23  Instance_1           1.0  ...   600.402           ok
  24  Instance_1           1.0  ...  3600.000       memout
  25  Instance_1           1.0  ...  3600.000       memout
  26  Instance_1           1.0  ...    78.121           ok
  27  Instance_1           1.0  ...   600.402           ok
  28  Instance_1           1.0  ...  3600.000       memout
  29  Instance_1           1.0  ...  3600.000       memout
  ..         ...           ...  ...       ...          ...
  66  Instance_4           1.0  ...   422.530           ok
  67  Instance_4           1.0  ...  3600.000      timeout
  68  Instance_4           1.0  ...  3600.000       memout
  69  Instance_4           1.0  ...  3600.000       memout
  70  Instance_4           1.0  ...   422.530           ok
  71  Instance_4           1.0  ...  3600.000      timeout
  72  Instance_4           1.0  ...  3600.000       memout
  73  Instance_4           1.0  ...  3600.000       memout
  74  Instance_4           1.0  ...   422.530           ok
  75  Instance_4           1.0  ...  3600.000      timeout
  76  Instance_4           1.0  ...  3600.000       memout
  77  Instance_4           1.0  ...  3600.000       memout
  78  Instance_4           1.0  ...   422.530           ok
  79  Instance_4           1.0  ...  3600.000      timeout
  80  Instance_5           1.0  ...   182.019           ok
  81  Instance_5           1.0  ...  3600.000       memout
  82  Instance_5           1.0  ...    66.988           ok
  83  Instance_5           1.0  ...   249.488           ok
  84  Instance_5           1.0  ...   182.019           ok
  85  Instance_5           1.0  ...  3600.000       memout
  86  Instance_5           1.0  ...    66.988           ok
  87  Instance_5           1.0  ...   249.488           ok
  88  Instance_5           1.0  ...   182.019           ok
  89  Instance_5           1.0  ...  3600.000       memout
  90  Instance_5           1.0  ...    66.988           ok
  91  Instance_5           1.0  ...   249.488           ok
  92  Instance_5           1.0  ...   182.019           ok
  93  Instance_5           1.0  ...  3600.000       memout
  94  Instance_5           1.0  ...    66.988           ok
  95  Instance_5           1.0  ...   249.488           ok
  
  [96 rows x 9 columns],
  hconcat: [VConcatChart({
    vconcat: [Chart({
      encoding: FacetedEncoding({
        color: ColorValue({
          condition: Color({
            field: FieldName('algorithm_y'),
            legend: None,
            scale: Scale({
              domain: ['algorithm_0', 'algorithm_1', 'i

In [0]:
def save_scatter(chart, name='chart'):
  chart.save(name+'.html')

In [0]:
sample_scatter = create_comparative_scatter(alg_joined, 'instance_id', 'algorithm_x', 'algorithm_y', 'runtime_x', 'runtime_y', 
                                            'runstatus_x', 'runstatus_y', four_colors)

In [32]:
sample_scatter

HConcatChart({
  data:    instance_id  repetition_x  ... runtime_y  runstatus_y
  0   Instance_0           1.0  ...  3600.000       memout
  1   Instance_0           1.0  ...  3600.000       memout
  2   Instance_0           1.0  ...   311.863           ok
  3   Instance_0           1.0  ...  3600.000      timeout
  4   Instance_0           1.0  ...  3600.000       memout
  5   Instance_0           1.0  ...  3600.000       memout
  6   Instance_0           1.0  ...   311.863           ok
  7   Instance_0           1.0  ...  3600.000      timeout
  8   Instance_0           1.0  ...  3600.000       memout
  9   Instance_0           1.0  ...  3600.000       memout
  10  Instance_0           1.0  ...   311.863           ok
  11  Instance_0           1.0  ...  3600.000      timeout
  12  Instance_0           1.0  ...  3600.000       memout
  13  Instance_0           1.0  ...  3600.000       memout
  14  Instance_0           1.0  ...   311.863           ok
  15  Instance_0           1.0  ...  3600.000      timeout
  16  Instance_1           1.0  ...  3600.000       memout
  17  Instance_1           1.0  ...  3600.000       memout
  18  Instance_1           1.0  ...    78.121           ok
  19  Instance_1           1.0  ...   600.402           ok
  20  Instance_1           1.0  ...  3600.000       memout
  21  Instance_1           1.0  ...  3600.000       memout
  22  Instance_1           1.0  ...    78.121           ok
  23  Instance_1           1.0  ...   600.402           ok
  24  Instance_1           1.0  ...  3600.000       memout
  25  Instance_1           1.0  ...  3600.000       memout
  26  Instance_1           1.0  ...    78.121           ok
  27  Instance_1           1.0  ...   600.402           ok
  28  Instance_1           1.0  ...  3600.000       memout
  29  Instance_1           1.0  ...  3600.000       memout
  ..         ...           ...  ...       ...          ...
  66  Instance_4           1.0  ...   422.530           ok
  67  Instance_4           1.0  ...  3600.000      timeout
  68  Instance_4           1.0  ...  3600.000       memout
  69  Instance_4           1.0  ...  3600.000       memout
  70  Instance_4           1.0  ...   422.530           ok
  71  Instance_4           1.0  ...  3600.000      timeout
  72  Instance_4           1.0  ...  3600.000       memout
  73  Instance_4           1.0  ...  3600.000       memout
  74  Instance_4           1.0  ...   422.530           ok
  75  Instance_4           1.0  ...  3600.000      timeout
  76  Instance_4           1.0  ...  3600.000       memout
  77  Instance_4           1.0  ...  3600.000       memout
  78  Instance_4           1.0  ...   422.530           ok
  79  Instance_4           1.0  ...  3600.000      timeout
  80  Instance_5           1.0  ...   182.019           ok
  81  Instance_5           1.0  ...  3600.000       memout
  82  Instance_5           1.0  ...    66.988           ok
  83  Instance_5           1.0  ...   249.488           ok
  84  Instance_5           1.0  ...   182.019           ok
  85  Instance_5           1.0  ...  3600.000       memout
  86  Instance_5           1.0  ...    66.988           ok
  87  Instance_5           1.0  ...   249.488           ok
  88  Instance_5           1.0  ...   182.019           ok
  89  Instance_5           1.0  ...  3600.000       memout
  90  Instance_5           1.0  ...    66.988           ok
  91  Instance_5           1.0  ...   249.488           ok
  92  Instance_5           1.0  ...   182.019           ok
  93  Instance_5           1.0  ...  3600.000       memout
  94  Instance_5           1.0  ...    66.988           ok
  95  Instance_5           1.0  ...   249.488           ok
  
  [96 rows x 9 columns],
  hconcat: [VConcatChart({
    vconcat: [Chart({
      encoding: FacetedEncoding({
        color: ColorValue({
          condition: Color({
            field: FieldName('algorithm_y'),
            legend: None,
            scale: Scale({
              domain: ['algorithm_0', 'algorithm_1', 'i

In [0]:
save_scatter(sample_scatter, 'sample scatter')

In [34]:
tester = make_graph(alg_joined, 'runtime_x', 'runtime_y')
tester

Chart({
  data:    instance_id  repetition_x  ... runtime_y  runstatus_y
  0   Instance_0           1.0  ...  3600.000       memout
  1   Instance_0           1.0  ...  3600.000       memout
  2   Instance_0           1.0  ...   311.863           ok
  3   Instance_0           1.0  ...  3600.000      timeout
  4   Instance_0           1.0  ...  3600.000       memout
  5   Instance_0           1.0  ...  3600.000       memout
  6   Instance_0           1.0  ...   311.863           ok
  7   Instance_0           1.0  ...  3600.000      timeout
  8   Instance_0           1.0  ...  3600.000       memout
  9   Instance_0           1.0  ...  3600.000       memout
  10  Instance_0           1.0  ...   311.863           ok
  11  Instance_0           1.0  ...  3600.000      timeout
  12  Instance_0           1.0  ...  3600.000       memout
  13  Instance_0           1.0  ...  3600.000       memout
  14  Instance_0           1.0  ...   311.863           ok
  15  Instance_0           1.0  ...  3600.000      timeout
  16  Instance_1           1.0  ...  3600.000       memout
  17  Instance_1           1.0  ...  3600.000       memout
  18  Instance_1           1.0  ...    78.121           ok
  19  Instance_1           1.0  ...   600.402           ok
  20  Instance_1           1.0  ...  3600.000       memout
  21  Instance_1           1.0  ...  3600.000       memout
  22  Instance_1           1.0  ...    78.121           ok
  23  Instance_1           1.0  ...   600.402           ok
  24  Instance_1           1.0  ...  3600.000       memout
  25  Instance_1           1.0  ...  3600.000       memout
  26  Instance_1           1.0  ...    78.121           ok
  27  Instance_1           1.0  ...   600.402           ok
  28  Instance_1           1.0  ...  3600.000       memout
  29  Instance_1           1.0  ...  3600.000       memout
  ..         ...           ...  ...       ...          ...
  66  Instance_4           1.0  ...   422.530           ok
  67  Instance_4           1.0  ...  3600.000      timeout
  68  Instance_4           1.0  ...  3600.000       memout
  69  Instance_4           1.0  ...  3600.000       memout
  70  Instance_4           1.0  ...   422.530           ok
  71  Instance_4           1.0  ...  3600.000      timeout
  72  Instance_4           1.0  ...  3600.000       memout
  73  Instance_4           1.0  ...  3600.000       memout
  74  Instance_4           1.0  ...   422.530           ok
  75  Instance_4           1.0  ...  3600.000      timeout
  76  Instance_4           1.0  ...  3600.000       memout
  77  Instance_4           1.0  ...  3600.000       memout
  78  Instance_4           1.0  ...   422.530           ok
  79  Instance_4           1.0  ...  3600.000      timeout
  80  Instance_5           1.0  ...   182.019           ok
  81  Instance_5           1.0  ...  3600.000       memout
  82  Instance_5           1.0  ...    66.988           ok
  83  Instance_5           1.0  ...   249.488           ok
  84  Instance_5           1.0  ...   182.019           ok
  85  Instance_5           1.0  ...  3600.000       memout
  86  Instance_5           1.0  ...    66.988           ok
  87  Instance_5           1.0  ...   249.488           ok
  88  Instance_5           1.0  ...   182.019           ok
  89  Instance_5           1.0  ...  3600.000       memout
  90  Instance_5           1.0  ...    66.988           ok
  91  Instance_5           1.0  ...   249.488           ok
  92  Instance_5           1.0  ...   182.019           ok
  93  Instance_5           1.0  ...  3600.000       memout
  94  Instance_5           1.0  ...    66.988           ok
  95  Instance_5           1.0  ...   249.488           ok
  
  [96 rows x 9 columns],
  encoding: FacetedEncoding({
    x: X({
      field: 'runtime_x',
      type: 'quantitative'
    }),
    y: Y({
      field: 'runtime_y',
      type: 'quantitative'
    })
  }),
  mark: MarkDef({
    size: 60,
    type: 'circle'
  })
})

In [0]:
# url = 'alg_joined.json'
# alg_joined.to_json(url, orient='records')

# json_scatter = create_comparative_scatter(url, 'instance_id', 'algorithm_x', 'algorithm_y', 'runtime_x', 'runtime_y', 
#                                             'runstatus_x', 'runstatus_y', four_colors)

In [37]:
test_tip = tester.encode(tooltip = ['instance_id'])
test_tip

Chart({
  data:    instance_id  repetition_x  ... runtime_y  runstatus_y
  0   Instance_0           1.0  ...  3600.000       memout
  1   Instance_0           1.0  ...  3600.000       memout
  2   Instance_0           1.0  ...   311.863           ok
  3   Instance_0           1.0  ...  3600.000      timeout
  4   Instance_0           1.0  ...  3600.000       memout
  5   Instance_0           1.0  ...  3600.000       memout
  6   Instance_0           1.0  ...   311.863           ok
  7   Instance_0           1.0  ...  3600.000      timeout
  8   Instance_0           1.0  ...  3600.000       memout
  9   Instance_0           1.0  ...  3600.000       memout
  10  Instance_0           1.0  ...   311.863           ok
  11  Instance_0           1.0  ...  3600.000      timeout
  12  Instance_0           1.0  ...  3600.000       memout
  13  Instance_0           1.0  ...  3600.000       memout
  14  Instance_0           1.0  ...   311.863           ok
  15  Instance_0           1.0  ...  3600.000      timeout
  16  Instance_1           1.0  ...  3600.000       memout
  17  Instance_1           1.0  ...  3600.000       memout
  18  Instance_1           1.0  ...    78.121           ok
  19  Instance_1           1.0  ...   600.402           ok
  20  Instance_1           1.0  ...  3600.000       memout
  21  Instance_1           1.0  ...  3600.000       memout
  22  Instance_1           1.0  ...    78.121           ok
  23  Instance_1           1.0  ...   600.402           ok
  24  Instance_1           1.0  ...  3600.000       memout
  25  Instance_1           1.0  ...  3600.000       memout
  26  Instance_1           1.0  ...    78.121           ok
  27  Instance_1           1.0  ...   600.402           ok
  28  Instance_1           1.0  ...  3600.000       memout
  29  Instance_1           1.0  ...  3600.000       memout
  ..         ...           ...  ...       ...          ...
  66  Instance_4           1.0  ...   422.530           ok
  67  Instance_4           1.0  ...  3600.000      timeout
  68  Instance_4           1.0  ...  3600.000       memout
  69  Instance_4           1.0  ...  3600.000       memout
  70  Instance_4           1.0  ...   422.530           ok
  71  Instance_4           1.0  ...  3600.000      timeout
  72  Instance_4           1.0  ...  3600.000       memout
  73  Instance_4           1.0  ...  3600.000       memout
  74  Instance_4           1.0  ...   422.530           ok
  75  Instance_4           1.0  ...  3600.000      timeout
  76  Instance_4           1.0  ...  3600.000       memout
  77  Instance_4           1.0  ...  3600.000       memout
  78  Instance_4           1.0  ...   422.530           ok
  79  Instance_4           1.0  ...  3600.000      timeout
  80  Instance_5           1.0  ...   182.019           ok
  81  Instance_5           1.0  ...  3600.000       memout
  82  Instance_5           1.0  ...    66.988           ok
  83  Instance_5           1.0  ...   249.488           ok
  84  Instance_5           1.0  ...   182.019           ok
  85  Instance_5           1.0  ...  3600.000       memout
  86  Instance_5           1.0  ...    66.988           ok
  87  Instance_5           1.0  ...   249.488           ok
  88  Instance_5           1.0  ...   182.019           ok
  89  Instance_5           1.0  ...  3600.000       memout
  90  Instance_5           1.0  ...    66.988           ok
  91  Instance_5           1.0  ...   249.488           ok
  92  Instance_5           1.0  ...   182.019           ok
  93  Instance_5           1.0  ...  3600.000       memout
  94  Instance_5           1.0  ...    66.988           ok
  95  Instance_5           1.0  ...   249.488           ok
  
  [96 rows x 9 columns],
  encoding: FacetedEncoding({
    tooltip: [Tooltip({
      field: 'instance_id',
      type: 'nominal'
    })],
    x: X({
      field: 'runtime_x',
      type: 'quantitative'
    }),
    y: Y({
      field: 'runtime_y',
      type: 'quantitative'
    })
  }),
  mark: MarkDef({
    size: 60,
    ty

In [38]:
corgi_in = pd.read_csv('corgi_named.csv', index_col = None)

corgi_chart = alt.Chart(corgi_in).mark_bar().encode(
    x = 'Name',
    y = 'Cuteness',
    color = alt.Color('Name', legend=None)
)

corgi_chart

Chart({
  data:                       Name  ... Arbitrary numbers 1-5
  0                    Holly  ...                     1
  1                    Ralph  ...                     2
  2  Molly ("Thing of Evil")  ...                     3
  3                      Ein  ...                     4
  4                     Loki  ...                     5
  5                    Rufus  ...                     1
  6             Sutter Brown  ...                     2
  7               Hope Corgi  ...                     3
  
  [8 rows x 5 columns],
  encoding: FacetedEncoding({
    color: Color({
      field: 'Name',
      legend: None,
      type: 'nominal'
    }),
    x: X({
      field: 'Name',
      type: 'nominal'
    }),
    y: Y({
      field: 'Cuteness',
      type: 'quantitative'
    })
  }),
  mark: 'bar'
})

In [0]:
# url = 'corgi_in.json'
# corgi_in.to_json(url, orient = 'records')

# corgi_chart_from_url = alt.Chart(url).mark_line().encode(
#     x = 'Name:N',
#     y = 'Cuteness:Q'
# )

# corgi_chart_from_url

In [40]:
import altair as alt
from vega_datasets import data
url = data.cars.url

alt.Chart(url).mark_point().encode(
    x='Horsepower:Q',
    y='Miles_per_Gallon:Q'
)

Chart({
  data: 'https://vega.github.io/vega-datasets/data/cars.json',
  encoding: FacetedEncoding({
    x: X({
      field: 'Horsepower',
      type: 'quantitative'
    }),
    y: Y({
      field: 'Miles_per_Gallon',
      type: 'quantitative'
    })
  }),
  mark: 'point'
})

In [0]:
# file_name = 'corgi_in.json'

# alt.Chart(file_name).mark_point().encode(
#     x='Name:N',
#     y = 'Cuteness:Q'
# )

In [42]:
data = alt.Data(values=[{'x': 'A', 'y': 5},
                        {'x': 'B', 'y': 3},
                        {'x': 'C', 'y': 6},
                        {'x': 'D', 'y': 7},
                        {'x': 'E', 'y': 2}])
alt.Chart(data).mark_bar().encode(
    x='x:O',  # specify ordinal data
    y='y:Q',  # specify quantitative data
)

Chart({
  data: Data({
    values: [{'x': 'A', 'y': 5}, {'x': 'B', 'y': 3}, {'x': 'C', 'y': 6}, {'x': 'D', 'y': 7}, {'x': 'E', 'y': 2}]
  }),
  encoding: FacetedEncoding({
    x: X({
      field: 'x',
      type: 'ordinal'
    }),
    y: Y({
      field: 'y',
      type: 'quantitative'
    })
  }),
  mark: 'bar'
})

In [43]:
corgis = [
    {"Name":"Holly","Source":"Queen of England", 
     "Is this Real or Not Real?":"Real","Cuteness":7},
    {"Name":"Ralph","Source":"Instagram",
     "Is this Real or Not Real?":"Real","Cuteness":7},
    {"Name":"Molly (\"Thing of Evil\")","Source":"Stephen King",
     "Is this Real or Not Real?":"Real","Cuteness":9},
    {"Name":"Ein","Source":"Cowboy Bebop",
     "Is this Real or Not Real?":"Not Real","Cuteness":11},
    {"Name":"Loki","Source":"Instagram",
     "Is this Real or Not Real?":"Real","Cuteness":8},
    {"Name":"Rufus","Source":"Amazon.com",
     "Is this Real or Not Real?":"Real","Cuteness":9},
    {"Name":"Sutter Brown","Source":"Jerry Brown (gov of Cali)",
     "Is this Real or Not Real?":"Real","Cuteness":8},
    {"Name":"Hope Corgi","Source":"DC Comics",
     "Is this Real or Not Real?":"Not Real","Cuteness":11}]

corgi_alt_data = alt.Data(values = corgis)

alt.Chart(corgi_alt_data).mark_bar().encode(
    x = 'Name:N',
    y = 'Cuteness:Q',
    color = alt.Color('Name:N', legend=None)
)

Chart({
  data: Data({
    values: [{'Name': 'Holly', 'Source': 'Queen of England', 'Is this Real or Not Real?': 'Real', 'Cuteness': 7}, {'Name': 'Ralph', 'Source': 'Instagram', 'Is this Real or Not Real?': 'Real', 'Cuteness': 7}, {'Name': 'Molly ("Thing of Evil")', 'Source': 'Stephen King', 'Is this Real or Not Real?': 'Real', 'Cuteness': 9}, {'Name': 'Ein', 'Source': 'Cowboy Bebop', 'Is this Real or Not Real?': 'Not Real', 'Cuteness': 11}, {'Name': 'Loki', 'Source': 'Instagram', 'Is this Real or Not Real?': 'Real', 'Cuteness': 8}, {'Name': 'Rufus', 'Source': 'Amazon.com', 'Is this Real or Not Real?': 'Real', 'Cuteness': 9}, {'Name': 'Sutter Brown', 'Source': 'Jerry Brown (gov of Cali)', 'Is this Real or Not Real?': 'Real', 'Cuteness': 8}, {'Name': 'Hope Corgi', 'Source': 'DC Comics', 'Is this Real or Not Real?': 'Not Real', 'Cuteness': 11}]
  }),
  encoding: FacetedEncoding({
    color: Color({
      field: 'Name',
      legend: None,
      type: 'nominal'
    }),
    x: X({
      field: 'Name',
      type: 'nominal'
    }),
    y: Y({
      field: 'Cuteness',
      type: 'quantitative'
    })
  }),
  mark: 'bar'
})

In [0]:
# corgi_from_json = alt.Data.from_json(corgis)

# corgi_from_json

In [46]:
import altair as alt
import pandas as pd
data = pd.DataFrame({'x': [1, 2, 3], 'y': [2, 1, 2]})

chart = alt.Chart(data).mark_line().encode(
     x='x',
     y='y'
)

from pprint import pprint
pprint(chart.to_dict())

chart

{'$schema': 'https://vega.github.io/schema/vega-lite/v3.3.0.json',
 'config': {'mark': {'tooltip': None}, 'view': {'height': 300, 'width': 400}},
 'data': {'name': 'data-2b857397a15d9ff67a8419d09d5c73c6'},
 'datasets': {'data-2b857397a15d9ff67a8419d09d5c73c6': [{'x': 1, 'y': 2},
                                                        {'x': 2, 'y': 1},
                                                        {'x': 3, 'y': 2}]},
 'encoding': {'x': {'field': 'x', 'type': 'quantitative'},
              'y': {'field': 'y', 'type': 'quantitative'}},
 'mark': 'line'}


Chart({
  data:    x  y
  0  1  2
  1  2  1
  2  3  2,
  encoding: FacetedEncoding({
    x: X({
      field: 'x',
      type: 'quantitative'
    }),
    y: Y({
      field: 'y',
      type: 'quantitative'
    })
  }),
  mark: 'line'
})

In [47]:
#alt.data_transformers.enable('json')

url = 'data.json'
data.to_json(url, orient='records')

chart = alt.Chart(url).mark_line().encode(
    x='x:Q',
    y='y:Q'
)
pprint(chart.to_dict())

chart

{'$schema': 'https://vega.github.io/schema/vega-lite/v3.3.0.json',
 'config': {'mark': {'tooltip': None}, 'view': {'height': 300, 'width': 400}},
 'data': {'url': 'data.json'},
 'encoding': {'x': {'field': 'x', 'type': 'quantitative'},
              'y': {'field': 'y', 'type': 'quantitative'}},
 'mark': 'line'}


Chart({
  data: 'data.json',
  encoding: FacetedEncoding({
    x: X({
      field: 'x',
      type: 'quantitative'
    }),
    y: Y({
      field: 'y',
      type: 'quantitative'
    })
  }),
  mark: 'line'
})

In [48]:
c_data = pd.read_csv('corgi_named.csv', index_col = None)
c_data

,Name,Source,Is this Real or Not Real?,Cuteness,Arbitrary numbers 1-5
0,Holly,Queen of England,Real,7,1
1,Ralph,Instagram,Real,7,2
2,"Molly (""Thing of Evil"")",Stephen King,Real,9,3
3,Ein,Cowboy Bebop,Not Real,11,4
4,Loki,Instagram,Real,8,5
5,Rufus,Amazon.com,Real,9,1
6,Sutter Brown,Jerry Brown (gov of Cali),Real,8,2
7,Hope Corgi,DC Comics,Not Real,11,3


In [0]:
# c_chart = alt.Chart(corgi_data).mark_bar().encode(
#     x = 'Name',
#     y = 'Cuteness'
# )

# pprint(c_chart.to_dict())

In [51]:
url = 'c_data.json'
c_data.to_json(url, orient = 'records')

c_chart = alt.Chart(url).mark_bar().encode(
    x = 'Name:N',
    y = 'Cuteness:Q'
)

pprint(c_chart.to_dict())

{'$schema': 'https://vega.github.io/schema/vega-lite/v3.3.0.json',
 'config': {'mark': {'tooltip': None}, 'view': {'height': 300, 'width': 400}},
 'data': {'url': 'c_data.json'},
 'encoding': {'x': {'field': 'Name', 'type': 'nominal'},
              'y': {'field': 'Cuteness', 'type': 'quantitative'}},
 'mark': 'bar'}


test basic bar chart, large data, not joined

In [0]:
simple_bar_graph = make_graph(algSource, 'algorithm', 'runtime', 'bar')

In [53]:
simple_bar_graph

Chart({
  data:    instance_id  repetition       algorithm   runtime runstatus
  0   Instance_0         1.0     algorithm_0  3600.000    memout
  1   Instance_0         1.0     algorithm_1  3600.000    memout
  2   Instance_0         1.0  id_algorithm_0   311.863        ok
  3   Instance_0         1.0  id_algorithm_1  3600.000   timeout
  4   Instance_1         1.0     algorithm_0  3600.000    memout
  5   Instance_1         1.0     algorithm_1  3600.000    memout
  6   Instance_1         1.0  id_algorithm_0    78.121        ok
  7   Instance_1         1.0  id_algorithm_1   600.402        ok
  8   Instance_2         1.0     algorithm_0  3600.000    memout
  9   Instance_2         1.0     algorithm_1  3600.000    memout
  10  Instance_2         1.0  id_algorithm_0     0.012        ok
  11  Instance_2         1.0  id_algorithm_1    60.720        ok
  12  Instance_3         1.0     algorithm_0  3600.000    memout
  13  Instance_3         1.0     algorithm_1    92.986        ok
  14  Instance_3         1.0  id_algorithm_0  2031.039        ok
  15  Instance_3         1.0  id_algorithm_1   141.857        ok
  16  Instance_4         1.0     algorithm_0  3600.000    memout
  17  Instance_4         1.0     algorithm_1  3600.000    memout
  18  Instance_4         1.0  id_algorithm_0   422.530        ok
  19  Instance_4         1.0  id_algorithm_1  3600.000   timeout
  20  Instance_5         1.0     algorithm_0   182.019        ok
  21  Instance_5         1.0     algorithm_1  3600.000    memout
  22  Instance_5         1.0  id_algorithm_0    66.988        ok
  23  Instance_5         1.0  id_algorithm_1   249.488        ok,
  encoding: FacetedEncoding({
    x: X({
      field: 'algorithm',
      type: 'nominal'
    }),
    y: Y({
      field: 'runtime',
      type: 'quantitative'
    })
  }),
  mark: 'bar'
})

In [54]:
algSource.head()

,instance_id,repetition,algorithm,runtime,runstatus
0,Instance_0,1.0,algorithm_0,3600.000,memout
1,Instance_0,1.0,algorithm_1,3600.000,memout
2,Instance_0,1.0,id_algorithm_0,311.863,ok
3,Instance_0,1.0,id_algorithm_1,3600.000,timeout
4,Instance_1,1.0,algorithm_0,3600.000,memout


In [0]:
# alt.Chart(algSource).mark_circle().encode(
#     x = 'runtime:Q',
#     y = 'runtime_y:Q'
# ).transform_lookup(
#     lookup='comparison',
#     from_=alt.LookupData(data=algSource, key = 'instance_id',
#                         fields = as('runtime'))#???
# )

practice joinable tables (based on characters from The Dresden Chronicles by Jim Butcher)

In [0]:
friend = {
    'name': ['karrin murphy', 'thomas raith', 'susan rodriguez', 
             'molly carpenter', 'major general minimus toot-toot', 'lady lily',
             'elaine mallory'],
    
    'monster_type': ['human', 'white court vampire', 'red court vampire',
                     'warlock', 'fae', 'fae', 'warlock'],
    
    'life_status': ['living', 'living', 'dead', 'living', 'living', 'dead', 
                    'living'],
    
    'favoritism_points': [10, 8, 7, 5, 3, 2, 1]
}

monster = {
    'type': ['human', 'white court vampire', 'red court vampire', 
                     'warlock', 'fae'],
    
    'danger_points': [2, 6, 5, 5, 10]
}

friends = pd.DataFrame(friend)
monsters = pd.DataFrame(monster)

In [58]:
alt.Chart(friends).mark_bar().encode(
    x = 'name:N',
    y = 'favoritism_points:Q'
)

Chart({
  data:                               name  ... favoritism_points
  0                    karrin murphy  ...                10
  1                     thomas raith  ...                 8
  2                  susan rodriguez  ...                 7
  3                  molly carpenter  ...                 5
  4  major general minimus toot-toot  ...                 3
  5                        lady lily  ...                 2
  6                   elaine mallory  ...                 1
  
  [7 rows x 4 columns],
  encoding: FacetedEncoding({
    x: X({
      field: 'name',
      type: 'nominal'
    }),
    y: Y({
      field: 'favoritism_points',
      type: 'quantitative'
    })
  }),
  mark: 'bar'
})

In [59]:
fave_vs_danger = alt.Chart(friends).mark_circle(size=100).encode(
    x = 'danger:Q',
    y = 'favoritism_points:Q',
    tooltip = ['name', 'favoritism_points', 'monster_type', 'danger:Q'],
    color = alt.Color('monster_type')
).transform_lookup(
    lookup = 'monster_type',
    from_ = alt.LookupData(data=monsters, key = 'type',
                           fields = ['danger_points']),
    as_ = ['danger']
)

fave_vs_danger

Chart({
  data:                               name  ... favoritism_points
  0                    karrin murphy  ...                10
  1                     thomas raith  ...                 8
  2                  susan rodriguez  ...                 7
  3                  molly carpenter  ...                 5
  4  major general minimus toot-toot  ...                 3
  5                        lady lily  ...                 2
  6                   elaine mallory  ...                 1
  
  [7 rows x 4 columns],
  encoding: FacetedEncoding({
    color: Color({
      field: 'monster_type',
      type: 'nominal'
    }),
    tooltip: [Tooltip({
      field: 'name',
      type: 'nominal'
    }), Tooltip({
      field: 'favoritism_points',
      type: 'quantitative'
    }), Tooltip({
      field: 'monster_type',
      type: 'nominal'
    }), Tooltip({
      field: 'danger',
      type: 'quantitative'
    })],
    x: X({
      field: 'danger',
      type: 'quantitative'
    }),
    y: Y({
      field: 'favoritism_points',
      type: 'quantitative'
    })
  }),
  mark: MarkDef({
    size: 100,
    type: 'circle'
  }),
  transform: [LookupTransform({
    as: ['danger'],
    from: LookupData({
      data:                   type  danger_points
      0                human              2
      1  white court vampire              6
      2    red court vampire              5
      3              warlock              5
      4                  fae             10,
      fields: ['danger_points'],
      key: 'type'
    }),
    lookup: 'monster_type'
  })]
})

In [0]:
from vega_datasets import data
people = data.lookup_people()
groups = data.lookup_groups()

In [61]:
groups.head()

,group,person
0,1,Alan
1,1,George
2,1,Fred
3,2,Steve
4,2,Nick


In [62]:
people.head()

,name,age,height
0,Alan,25,180
1,George,32,174
2,Fred,39,182
3,Steve,42,161
4,Nick,23,180


In [0]:
fave_vs_danger.save('fave_vs_danger.html')

In [0]:
fave_int = fave_vs_danger.interactive()
fave_int.save('fave_int.html')

In [65]:
pprint(fave_int.to_dict())

{'$schema': 'https://vega.github.io/schema/vega-lite/v3.3.0.json',
 'config': {'mark': {'tooltip': None}, 'view': {'height': 300, 'width': 400}},
 'data': {'name': 'data-cca94c3c58524e39a67bf5f8b150fdae'},
 'datasets': {'data-47489b3fdc0ce3ba2738c6de13bc98bd': [{'danger_points': 2,
                                                         'type': 'human'},
                                                        {'danger_points': 6,
                                                         'type': 'white court '
                                                                 'vampire'},
                                                        {'danger_points': 5,
                                                         'type': 'red court '
                                                                 'vampire'},
                                                        {'danger_points': 5,
                                                         'type': 'warlock'},
                      

In [66]:
algSource.head()

,instance_id,repetition,algorithm,runtime,runstatus
0,Instance_0,1.0,algorithm_0,3600.000,memout
1,Instance_0,1.0,algorithm_1,3600.000,memout
2,Instance_0,1.0,id_algorithm_0,311.863,ok
3,Instance_0,1.0,id_algorithm_1,3600.000,timeout
4,Instance_1,1.0,algorithm_0,3600.000,memout


In [67]:
alg_versus = alt.Chart(algSource).mark_circle(size=100).encode(
    x = 'runtime',
    y = 'runtime_y:Q',
    tooltip = ['instance_id', 'algorithm', 'runtime', 
               'algorithm_y:N', 'runtime_y:Q']
).transform_lookup(
    lookup = 'instance_id',
    from_ = alt.LookupData(algSource, 'instance_id', ['algorithm', 'runtime']),
    as_ = ['algorithm_y', 'runtime_y']
)

alg_versus
#pprint(alg_versus.to_dict())

Chart({
  data:    instance_id  repetition       algorithm   runtime runstatus
  0   Instance_0         1.0     algorithm_0  3600.000    memout
  1   Instance_0         1.0     algorithm_1  3600.000    memout
  2   Instance_0         1.0  id_algorithm_0   311.863        ok
  3   Instance_0         1.0  id_algorithm_1  3600.000   timeout
  4   Instance_1         1.0     algorithm_0  3600.000    memout
  5   Instance_1         1.0     algorithm_1  3600.000    memout
  6   Instance_1         1.0  id_algorithm_0    78.121        ok
  7   Instance_1         1.0  id_algorithm_1   600.402        ok
  8   Instance_2         1.0     algorithm_0  3600.000    memout
  9   Instance_2         1.0     algorithm_1  3600.000    memout
  10  Instance_2         1.0  id_algorithm_0     0.012        ok
  11  Instance_2         1.0  id_algorithm_1    60.720        ok
  12  Instance_3         1.0     algorithm_0  3600.000    memout
  13  Instance_3         1.0     algorithm_1    92.986        ok
  14  Instance_3         1.0  id_algorithm_0  2031.039        ok
  15  Instance_3         1.0  id_algorithm_1   141.857        ok
  16  Instance_4         1.0     algorithm_0  3600.000    memout
  17  Instance_4         1.0     algorithm_1  3600.000    memout
  18  Instance_4         1.0  id_algorithm_0   422.530        ok
  19  Instance_4         1.0  id_algorithm_1  3600.000   timeout
  20  Instance_5         1.0     algorithm_0   182.019        ok
  21  Instance_5         1.0     algorithm_1  3600.000    memout
  22  Instance_5         1.0  id_algorithm_0    66.988        ok
  23  Instance_5         1.0  id_algorithm_1   249.488        ok,
  encoding: FacetedEncoding({
    tooltip: [Tooltip({
      field: 'instance_id',
      type: 'nominal'
    }), Tooltip({
      field: 'algorithm',
      type: 'nominal'
    }), Tooltip({
      field: 'runtime',
      type: 'quantitative'
    }), Tooltip({
      field: 'algorithm_y',
      type: 'nominal'
    }), Tooltip({
      field: 'runtime_y',
      type: 'quantitative'
    })],
    x: X({
      field: 'runtime',
      type: 'quantitative'
    }),
    y: Y({
      field: 'runtime_y',
      type: 'quantitative'
    })
  }),
  mark: MarkDef({
    size: 100,
    type: 'circle'
  }),
  transform: [LookupTransform({
    as: ['algorithm_y', 'runtime_y'],
    from: LookupData({
      data:    instance_id  repetition       algorithm   runtime runstatus
      0   Instance_0         1.0     algorithm_0  3600.000    memout
      1   Instance_0         1.0     algorithm_1  3600.000    memout
      2   Instance_0         1.0  id_algorithm_0   311.863        ok
      3   Instance_0         1.0  id_algorithm_1  3600.000   timeout
      4   Instance_1         1.0     algorithm_0  3600.000    memout
      5   Instance_1         1.0     algorithm_1  3600.000    memout
      6   Instance_1         1.0  id_algorithm_0    78.121        ok
      7   Instance_1         1.0  id_algorithm_1   600.402        ok
      8   Instance_2         1.0     algorithm_0  3600.000    memout
      9   Instance_2         1.0     algorithm_1  3600.000    memout
      10  Instance_2         1.0  id_algorithm_0     0.012        ok
      11  Instance_2         1.0  id_algorithm_1    60.720        ok
      12  Instance_3         1.0     algorithm_0  3600.000    memout
      13  Instance_3         1.0     algorithm_1    92.986        ok
      14  Instance_3         1.0  id_algorithm_0  2031.039        ok
      15  Instance_3         1.0  id_algorithm_1   141.857        ok
      16  Instance_4         1.0     algorithm_0  3600.000    memout
      17  Instance_4         1.0     algorithm_1  3600.000    memout
      18  Instance_4         1.0  id_algorithm_0   422.530        ok
      19  Instance_4         1.0  id_algorithm_1  3600.000   timeout
      20  Instance_5         1.0     algorithm_0   182.019        ok
      21  Instance_5         1.0     algorithm_1  3600.000    memout
      22  Instance_5         1.0  id_algorithm_0    66.988        ok
      23  Ins

In [68]:
pprint(alg_versus.to_dict())

{'$schema': 'https://vega.github.io/schema/vega-lite/v3.3.0.json',
 'config': {'mark': {'tooltip': None}, 'view': {'height': 300, 'width': 400}},
 'data': {'name': 'data-c4275514345efec21b7aeb848c42e09b'},
 'datasets': {'data-c4275514345efec21b7aeb848c42e09b': [{'algorithm': 'algorithm_0',
                                                         'instance_id': 'Instance_0',
                                                         'repetition': 1.0,
                                                         'runstatus': 'memout',
                                                         'runtime': 3600.0},
                                                        {'algorithm': 'algorithm_1',
                                                         'instance_id': 'Instance_0',
                                                         'repetition': 1.0,
                                                         'runstatus': 'memout',
                                                         'runti

In [69]:
algSource.head()

,instance_id,repetition,algorithm,runtime,runstatus
0,Instance_0,1.0,algorithm_0,3600.000,memout
1,Instance_0,1.0,algorithm_1,3600.000,memout
2,Instance_0,1.0,id_algorithm_0,311.863,ok
3,Instance_0,1.0,id_algorithm_1,3600.000,timeout
4,Instance_1,1.0,algorithm_0,3600.000,memout


In [70]:
alt.Chart(algSource).mark_bar().encode(
    x = 'algorithm',
    y = 'runtime'
)

Chart({
  data:    instance_id  repetition       algorithm   runtime runstatus
  0   Instance_0         1.0     algorithm_0  3600.000    memout
  1   Instance_0         1.0     algorithm_1  3600.000    memout
  2   Instance_0         1.0  id_algorithm_0   311.863        ok
  3   Instance_0         1.0  id_algorithm_1  3600.000   timeout
  4   Instance_1         1.0     algorithm_0  3600.000    memout
  5   Instance_1         1.0     algorithm_1  3600.000    memout
  6   Instance_1         1.0  id_algorithm_0    78.121        ok
  7   Instance_1         1.0  id_algorithm_1   600.402        ok
  8   Instance_2         1.0     algorithm_0  3600.000    memout
  9   Instance_2         1.0     algorithm_1  3600.000    memout
  10  Instance_2         1.0  id_algorithm_0     0.012        ok
  11  Instance_2         1.0  id_algorithm_1    60.720        ok
  12  Instance_3         1.0     algorithm_0  3600.000    memout
  13  Instance_3         1.0     algorithm_1    92.986        ok
  14  Instance_3         1.0  id_algorithm_0  2031.039        ok
  15  Instance_3         1.0  id_algorithm_1   141.857        ok
  16  Instance_4         1.0     algorithm_0  3600.000    memout
  17  Instance_4         1.0     algorithm_1  3600.000    memout
  18  Instance_4         1.0  id_algorithm_0   422.530        ok
  19  Instance_4         1.0  id_algorithm_1  3600.000   timeout
  20  Instance_5         1.0     algorithm_0   182.019        ok
  21  Instance_5         1.0     algorithm_1  3600.000    memout
  22  Instance_5         1.0  id_algorithm_0    66.988        ok
  23  Instance_5         1.0  id_algorithm_1   249.488        ok,
  encoding: FacetedEncoding({
    x: X({
      field: 'algorithm',
      type: 'nominal'
    }),
    y: Y({
      field: 'runtime',
      type: 'quantitative'
    })
  }),
  mark: 'bar'
})

In [0]:
# alt.Chart(algSource).mark_circle(size=100).encode(
#     x = 'runtime',
#     y = 'runtime_y:Q',
#     tooltip = []
# )

In [0]:
#STUB
#creates scatter plot with the same column on both axes
#needs full interactivity and row lookup

def make_graph_comparison(source, lookup_key, column, name, feature, suffix, 
                          column_encoding = 'Q'):
  
  column_y = column + suffix + ':' + column_encoding
  name_y = name + suffix + ':N'
  feature_y = feature + suffix + ':N'
  
  chart = alt.Chart(source).mark_circle(size = 100).encode(
      x = column,
      y = column_y,
      tooltip = [lookup_key, column, name, feature, column_y, name_y, feature_y]
  ).transform_lookup(
      lookup = lookup_key,
      from_ = alt.LookupData(source, key = lookup_key,
                             fields = [column, name, feature]
                            ),
      as_ = [column + suffix, name + suffix, feature + suffix]
  )
  
  return chart

shows that not all of the data is retrieved by lookup. maybe i can make the key [id, alg]?

In [74]:
comp = make_graph_comparison(algSource, 'instance_id', 'runtime', 'algorithm', 
                             'runstatus', '_y')

comp_inst = comp.encode(color = 'instance_id:N')
comp_al_x = comp.encode(color = 'algorithm:N')
comp_al_y = comp.encode(color = 'algorithm_y:N')
comp_time_x = comp.encode(color = 'runtime:Q')
comp_time_y = comp.encode(color = 'runtime_y:Q')
comp_stat_x = comp.encode(color = 'runstatus:N')
comp_stat_y = comp.encode(color = 'runstatus_y:N')

comp_inst & (comp_al_x | comp_al_y) & (comp_time_x | comp_time_y) & (comp_stat_x | comp_stat_y)

VConcatChart({
  data:    instance_id  repetition       algorithm   runtime runstatus
  0   Instance_0         1.0     algorithm_0  3600.000    memout
  1   Instance_0         1.0     algorithm_1  3600.000    memout
  2   Instance_0         1.0  id_algorithm_0   311.863        ok
  3   Instance_0         1.0  id_algorithm_1  3600.000   timeout
  4   Instance_1         1.0     algorithm_0  3600.000    memout
  5   Instance_1         1.0     algorithm_1  3600.000    memout
  6   Instance_1         1.0  id_algorithm_0    78.121        ok
  7   Instance_1         1.0  id_algorithm_1   600.402        ok
  8   Instance_2         1.0     algorithm_0  3600.000    memout
  9   Instance_2         1.0     algorithm_1  3600.000    memout
  10  Instance_2         1.0  id_algorithm_0     0.012        ok
  11  Instance_2         1.0  id_algorithm_1    60.720        ok
  12  Instance_3         1.0     algorithm_0  3600.000    memout
  13  Instance_3         1.0     algorithm_1    92.986        ok
  14  Instance_3         1.0  id_algorithm_0  2031.039        ok
  15  Instance_3         1.0  id_algorithm_1   141.857        ok
  16  Instance_4         1.0     algorithm_0  3600.000    memout
  17  Instance_4         1.0     algorithm_1  3600.000    memout
  18  Instance_4         1.0  id_algorithm_0   422.530        ok
  19  Instance_4         1.0  id_algorithm_1  3600.000   timeout
  20  Instance_5         1.0     algorithm_0   182.019        ok
  21  Instance_5         1.0     algorithm_1  3600.000    memout
  22  Instance_5         1.0  id_algorithm_0    66.988        ok
  23  Instance_5         1.0  id_algorithm_1   249.488        ok,
  vconcat: [Chart({
    encoding: FacetedEncoding({
      color: Color({
        field: 'instance_id',
        type: 'nominal'
      }),
      tooltip: [Tooltip({
        field: 'instance_id',
        type: 'nominal'
      }), Tooltip({
        field: 'runtime',
        type: 'quantitative'
      }), Tooltip({
        field: 'algorithm',
        type: 'nominal'
      }), Tooltip({
        field: 'runstatus',
        type: 'nominal'
      }), Tooltip({
        field: 'runtime_y',
        type: 'quantitative'
      }), Tooltip({
        field: 'algorithm_y',
        type: 'nominal'
      }), Tooltip({
        field: 'runstatus_y',
        type: 'nominal'
      })],
      x: X({
        field: 'runtime',
        type: 'quantitative'
      }),
      y: Y({
        field: 'runtime_y',
        type: 'quantitative'
      })
    }),
    mark: MarkDef({
      size: 100,
      type: 'circle'
    }),
    transform: [LookupTransform({
      as: ['runtime_y', 'algorithm_y', 'runstatus_y'],
      from: LookupData({
        data:    instance_id  repetition       algorithm   runtime runstatus
        0   Instance_0         1.0     algorithm_0  3600.000    memout
        1   Instance_0         1.0     algorithm_1  3600.000    memout
        2   Instance_0         1.0  id_algorithm_0   311.863        ok
        3   Instance_0         1.0  id_algorithm_1  3600.000   timeout
        4   Instance_1         1.0     algorithm_0  3600.000    memout
        5   Instance_1         1.0     algorithm_1  3600.000    memout
        6   Instance_1         1.0  id_algorithm_0    78.121        ok
        7   Instance_1         1.0  id_algorithm_1   600.402        ok
        8   Instance_2         1.0     algorithm_0  3600.000    memout
        9   Instance_2         1.0     algorithm_1  3600.000    memout
        10  Instance_2         1.0  id_algorithm_0     0.012        ok
        11  Instance_2         1.0  id_algorithm_1    60.720        ok
        12  Instance_3         1.0     algorithm_0  3600.000    memout
        13  Instance_3         1.0     algorithm_1    92.986        ok
        14  Instance_3         1.0  id_algorithm_0  2031.039        ok
        15  Instance_3         1.0  id_algorithm_1   141.857        ok
        16  Instance_4         1.0     algorithm_0  3600.000    memout
        17  Instance_4         1.0     algorithm_1  

In [75]:
#  opts = list(data_source[column].unique())
#  dropdown = alt.binding_select(options = opts)
#  select = alt.selection_single(fields = [column],
#                                bind = dropdown,
#                                name = name_prefix
#                               )
 

opts = list(algSource['algorithm'].unique())
dropdown = alt.binding_select(options = opts)
select = alt.selection_single(fields = ['algorithm_y'],
                              bind = dropdown,
                              name = 'test'
                             )

comp_fil = comp.add_selection(select).transform_filter(select)
comp_fil

Chart({
  data:    instance_id  repetition       algorithm   runtime runstatus
  0   Instance_0         1.0     algorithm_0  3600.000    memout
  1   Instance_0         1.0     algorithm_1  3600.000    memout
  2   Instance_0         1.0  id_algorithm_0   311.863        ok
  3   Instance_0         1.0  id_algorithm_1  3600.000   timeout
  4   Instance_1         1.0     algorithm_0  3600.000    memout
  5   Instance_1         1.0     algorithm_1  3600.000    memout
  6   Instance_1         1.0  id_algorithm_0    78.121        ok
  7   Instance_1         1.0  id_algorithm_1   600.402        ok
  8   Instance_2         1.0     algorithm_0  3600.000    memout
  9   Instance_2         1.0     algorithm_1  3600.000    memout
  10  Instance_2         1.0  id_algorithm_0     0.012        ok
  11  Instance_2         1.0  id_algorithm_1    60.720        ok
  12  Instance_3         1.0     algorithm_0  3600.000    memout
  13  Instance_3         1.0     algorithm_1    92.986        ok
  14  Instance_3         1.0  id_algorithm_0  2031.039        ok
  15  Instance_3         1.0  id_algorithm_1   141.857        ok
  16  Instance_4         1.0     algorithm_0  3600.000    memout
  17  Instance_4         1.0     algorithm_1  3600.000    memout
  18  Instance_4         1.0  id_algorithm_0   422.530        ok
  19  Instance_4         1.0  id_algorithm_1  3600.000   timeout
  20  Instance_5         1.0     algorithm_0   182.019        ok
  21  Instance_5         1.0     algorithm_1  3600.000    memout
  22  Instance_5         1.0  id_algorithm_0    66.988        ok
  23  Instance_5         1.0  id_algorithm_1   249.488        ok,
  encoding: FacetedEncoding({
    tooltip: [Tooltip({
      field: 'instance_id',
      type: 'nominal'
    }), Tooltip({
      field: 'runtime',
      type: 'quantitative'
    }), Tooltip({
      field: 'algorithm',
      type: 'nominal'
    }), Tooltip({
      field: 'runstatus',
      type: 'nominal'
    }), Tooltip({
      field: 'runtime_y',
      type: 'quantitative'
    }), Tooltip({
      field: 'algorithm_y',
      type: 'nominal'
    }), Tooltip({
      field: 'runstatus_y',
      type: 'nominal'
    })],
    x: X({
      field: 'runtime',
      type: 'quantitative'
    }),
    y: Y({
      field: 'runtime_y',
      type: 'quantitative'
    })
  }),
  mark: MarkDef({
    size: 100,
    type: 'circle'
  }),
  selection: {'test': SelectionDef({
    bind: BindRadioSelect({
      input: 'select',
      options: ['algorithm_0', 'algorithm_1', 'id_algorithm_0', 'id_algorithm_1']
    }),
    fields: ['algorithm_y'],
    type: 'single'
  })},
  transform: [LookupTransform({
    as: ['runtime_y', 'algorithm_y', 'runstatus_y'],
    from: LookupData({
      data:    instance_id  repetition       algorithm   runtime runstatus
      0   Instance_0         1.0     algorithm_0  3600.000    memout
      1   Instance_0         1.0     algorithm_1  3600.000    memout
      2   Instance_0         1.0  id_algorithm_0   311.863        ok
      3   Instance_0         1.0  id_algorithm_1  3600.000   timeout
      4   Instance_1         1.0     algorithm_0  3600.000    memout
      5   Instance_1         1.0     algorithm_1  3600.000    memout
      6   Instance_1         1.0  id_algorithm_0    78.121        ok
      7   Instance_1         1.0  id_algorithm_1   600.402        ok
      8   Instance_2         1.0     algorithm_0  3600.000    memout
      9   Instance_2         1.0     algorithm_1  3600.000    memout
      10  Instance_2         1.0  id_algorithm_0     0.012        ok
      11  Instance_2         1.0  id_algorithm_1    60.720        ok
      12  Instance_3         1.0     algorithm_0  3600.000    memout
      13  Instance_3         1.0     algorithm_1    92.986        ok
      14  Instance_3         1.0  id_algorithm_0  2031.039        ok
      15  Instance_3         1.0  id_algorithm_1   141.857        ok
      16  Instance_4         1.0     algorithm_0  3600.000    memout
      17  Instance_4         1.0     algorith

In [76]:
algSource.head()

,instance_id,repetition,algorithm,runtime,runstatus
0,Instance_0,1.0,algorithm_0,3600.000,memout
1,Instance_0,1.0,algorithm_1,3600.000,memout
2,Instance_0,1.0,id_algorithm_0,311.863,ok
3,Instance_0,1.0,id_algorithm_1,3600.000,timeout
4,Instance_1,1.0,algorithm_0,3600.000,memout


In [77]:
alt.Chart(algSource).mark_bar().encode(
    x = 'algorithm',
    y = 'sum(runtime)'
)

Chart({
  data:    instance_id  repetition       algorithm   runtime runstatus
  0   Instance_0         1.0     algorithm_0  3600.000    memout
  1   Instance_0         1.0     algorithm_1  3600.000    memout
  2   Instance_0         1.0  id_algorithm_0   311.863        ok
  3   Instance_0         1.0  id_algorithm_1  3600.000   timeout
  4   Instance_1         1.0     algorithm_0  3600.000    memout
  5   Instance_1         1.0     algorithm_1  3600.000    memout
  6   Instance_1         1.0  id_algorithm_0    78.121        ok
  7   Instance_1         1.0  id_algorithm_1   600.402        ok
  8   Instance_2         1.0     algorithm_0  3600.000    memout
  9   Instance_2         1.0     algorithm_1  3600.000    memout
  10  Instance_2         1.0  id_algorithm_0     0.012        ok
  11  Instance_2         1.0  id_algorithm_1    60.720        ok
  12  Instance_3         1.0     algorithm_0  3600.000    memout
  13  Instance_3         1.0     algorithm_1    92.986        ok
  14  Instance_3         1.0  id_algorithm_0  2031.039        ok
  15  Instance_3         1.0  id_algorithm_1   141.857        ok
  16  Instance_4         1.0     algorithm_0  3600.000    memout
  17  Instance_4         1.0     algorithm_1  3600.000    memout
  18  Instance_4         1.0  id_algorithm_0   422.530        ok
  19  Instance_4         1.0  id_algorithm_1  3600.000   timeout
  20  Instance_5         1.0     algorithm_0   182.019        ok
  21  Instance_5         1.0     algorithm_1  3600.000    memout
  22  Instance_5         1.0  id_algorithm_0    66.988        ok
  23  Instance_5         1.0  id_algorithm_1   249.488        ok,
  encoding: FacetedEncoding({
    x: X({
      field: 'algorithm',
      type: 'nominal'
    }),
    y: Y({
      aggregate: 'sum',
      field: 'runtime',
      type: 'quantitative'
    })
  }),
  mark: 'bar'
})

In [78]:
alt.Chart(algSource).mark_circle(size = 100).encode(
    x = 'runtime',
    y = 'runtime_y:Q'
).transform_lookup(
    lookup = '\[\'instance_id\'\]',
    from_ = alt.LookupData(algSource, key = 'instance_id',
                           fields = ['runtime']
                          ),
    as_ = ['runtime_y']
)

Chart({
  data:    instance_id  repetition       algorithm   runtime runstatus
  0   Instance_0         1.0     algorithm_0  3600.000    memout
  1   Instance_0         1.0     algorithm_1  3600.000    memout
  2   Instance_0         1.0  id_algorithm_0   311.863        ok
  3   Instance_0         1.0  id_algorithm_1  3600.000   timeout
  4   Instance_1         1.0     algorithm_0  3600.000    memout
  5   Instance_1         1.0     algorithm_1  3600.000    memout
  6   Instance_1         1.0  id_algorithm_0    78.121        ok
  7   Instance_1         1.0  id_algorithm_1   600.402        ok
  8   Instance_2         1.0     algorithm_0  3600.000    memout
  9   Instance_2         1.0     algorithm_1  3600.000    memout
  10  Instance_2         1.0  id_algorithm_0     0.012        ok
  11  Instance_2         1.0  id_algorithm_1    60.720        ok
  12  Instance_3         1.0     algorithm_0  3600.000    memout
  13  Instance_3         1.0     algorithm_1    92.986        ok
  14  Instance_3         1.0  id_algorithm_0  2031.039        ok
  15  Instance_3         1.0  id_algorithm_1   141.857        ok
  16  Instance_4         1.0     algorithm_0  3600.000    memout
  17  Instance_4         1.0     algorithm_1  3600.000    memout
  18  Instance_4         1.0  id_algorithm_0   422.530        ok
  19  Instance_4         1.0  id_algorithm_1  3600.000   timeout
  20  Instance_5         1.0     algorithm_0   182.019        ok
  21  Instance_5         1.0     algorithm_1  3600.000    memout
  22  Instance_5         1.0  id_algorithm_0    66.988        ok
  23  Instance_5         1.0  id_algorithm_1   249.488        ok,
  encoding: FacetedEncoding({
    x: X({
      field: 'runtime',
      type: 'quantitative'
    }),
    y: Y({
      field: 'runtime_y',
      type: 'quantitative'
    })
  }),
  mark: MarkDef({
    size: 100,
    type: 'circle'
  }),
  transform: [LookupTransform({
    as: ['runtime_y'],
    from: LookupData({
      data:    instance_id  repetition       algorithm   runtime runstatus
      0   Instance_0         1.0     algorithm_0  3600.000    memout
      1   Instance_0         1.0     algorithm_1  3600.000    memout
      2   Instance_0         1.0  id_algorithm_0   311.863        ok
      3   Instance_0         1.0  id_algorithm_1  3600.000   timeout
      4   Instance_1         1.0     algorithm_0  3600.000    memout
      5   Instance_1         1.0     algorithm_1  3600.000    memout
      6   Instance_1         1.0  id_algorithm_0    78.121        ok
      7   Instance_1         1.0  id_algorithm_1   600.402        ok
      8   Instance_2         1.0     algorithm_0  3600.000    memout
      9   Instance_2         1.0     algorithm_1  3600.000    memout
      10  Instance_2         1.0  id_algorithm_0     0.012        ok
      11  Instance_2         1.0  id_algorithm_1    60.720        ok
      12  Instance_3         1.0     algorithm_0  3600.000    memout
      13  Instance_3         1.0     algorithm_1    92.986        ok
      14  Instance_3         1.0  id_algorithm_0  2031.039        ok
      15  Instance_3         1.0  id_algorithm_1   141.857        ok
      16  Instance_4         1.0     algorithm_0  3600.000    memout
      17  Instance_4         1.0     algorithm_1  3600.000    memout
      18  Instance_4         1.0  id_algorithm_0   422.530        ok
      19  Instance_4         1.0  id_algorithm_1  3600.000   timeout
      20  Instance_5         1.0     algorithm_0   182.019        ok
      21  Instance_5         1.0     algorithm_1  3600.000    memout
      22  Instance_5         1.0  id_algorithm_0    66.988        ok
      23  Instance_5         1.0  id_algorithm_1   249.488        ok,
      fields: ['runtime'],
      key: 'instance_id'
    }),
    lookup: "\\['instance_id'\\]"
  })]
})

maybe I need to use a multi-index after all, so the lookup will get all of the entries instead of overwriting them

In [79]:
algMulti = algSource.set_index(['instance_id', 'algorithm'])
algMulti.head()

repetition   runtime runstatus
instance_id algorithm                                     
Instance_0  algorithm_0            1.0  3600.000    memout
            algorithm_1            1.0  3600.000    memout
            id_algorithm_0         1.0   311.863        ok
            id_algorithm_1         1.0  3600.000   timeout
Instance_1  algorithm_0            1.0  3600.000    memout

In [80]:
alt.Chart(algMulti).mark_bar().encode(
    x = 'algorithm',
    y = 'sum(runtime)'
)

ValueError: ignored

Chart({
  data:                             repetition   runtime runstatus
  instance_id algorithm                                     
  Instance_0  algorithm_0            1.0  3600.000    memout
              algorithm_1            1.0  3600.000    memout
              id_algorithm_0         1.0   311.863        ok
              id_algorithm_1         1.0  3600.000   timeout
  Instance_1  algorithm_0            1.0  3600.000    memout
              algorithm_1            1.0  3600.000    memout
              id_algorithm_0         1.0    78.121        ok
              id_algorithm_1         1.0   600.402        ok
  Instance_2  algorithm_0            1.0  3600.000    memout
              algorithm_1            1.0  3600.000    memout
              id_algorithm_0         1.0     0.012        ok
              id_algorithm_1         1.0    60.720        ok
  Instance_3  algorithm_0            1.0  3600.000    memout
              algorithm_1            1.0    92.986        ok
          

In [0]:
algReset = algMulti.reset_index()

In [82]:
algReset.head()

,instance_id,algorithm,repetition,runtime,runstatus
0,Instance_0,algorithm_0,1.0,3600.000,memout
1,Instance_0,algorithm_1,1.0,3600.000,memout
2,Instance_0,id_algorithm_0,1.0,311.863,ok
3,Instance_0,id_algorithm_1,1.0,3600.000,timeout
4,Instance_1,algorithm_0,1.0,3600.000,memout


since multiindex isn't supported, i'll try adding another column filled with tuple values to act as the hierarchical key

In [0]:
algReset['key_col'] = algReset['instance_id']
algReset.head()

,instance_id,algorithm,repetition,runtime,runstatus,key_col
0,Instance_0,algorithm_0,1.0,3600.000,memout,Instance_0
1,Instance_0,algorithm_1,1.0,3600.000,memout,Instance_0
2,Instance_0,id_algorithm_0,1.0,311.863,ok,Instance_0
3,Instance_0,id_algorithm_1,1.0,3600.000,timeout,Instance_0
4,Instance_1,algorithm_0,1.0,3600.000,memout,Instance_1


In [0]:
key_series = pd.Series()
key_series.append([algReset['instance_id'].iteritems(),algReset['algorithm']])

TypeError: ignored

https://www.geeksforgeeks.org/python-merge-two-lists-into-list-of-tuples/

In [0]:
key_series= pd.Series(map(lambda x, y: (x,y), algReset['instance_id'], algReset['algorithm']))
key_series

0        (Instance_0, algorithm_0)
1        (Instance_0, algorithm_1)
2     (Instance_0, id_algorithm_0)
3     (Instance_0, id_algorithm_1)
4        (Instance_1, algorithm_0)
5        (Instance_1, algorithm_1)
6     (Instance_1, id_algorithm_0)
7     (Instance_1, id_algorithm_1)
8        (Instance_2, algorithm_0)
9        (Instance_2, algorithm_1)
10    (Instance_2, id_algorithm_0)
11    (Instance_2, id_algorithm_1)
12       (Instance_3, algorithm_0)
13       (Instance_3, algorithm_1)
14    (Instance_3, id_algorithm_0)
15    (Instance_3, id_algorithm_1)
16       (Instance_4, algorithm_0)
17       (Instance_4, algorithm_1)
18    (Instance_4, id_algorithm_0)
19    (Instance_4, id_algorithm_1)
20       (Instance_5, algorithm_0)
21       (Instance_5, algorithm_1)
22    (Instance_5, id_algorithm_0)
23    (Instance_5, id_algorithm_1)
dtype: object

In [0]:
algReset['key_col'] = key_series
algReset

,instance_id,algorithm,repetition,runtime,runstatus,key_col
0,Instance_0,algorithm_0,1.0,3600.000,memout,"(Instance_0, algorithm_0)"
1,Instance_0,algorithm_1,1.0,3600.000,memout,"(Instance_0, algorithm_1)"
2,Instance_0,id_algorithm_0,1.0,311.863,ok,"(Instance_0, id_algorithm_0)"
3,Instance_0,id_algorithm_1,1.0,3600.000,timeout,"(Instance_0, id_algorithm_1)"
4,Instance_1,algorithm_0,1.0,3600.000,memout,"(Instance_1, algorithm_0)"
5,Instance_1,algorithm_1,1.0,3600.000,memout,"(Instance_1, algorithm_1)"
6,Instance_1,id_algorithm_0,1.0,78.121,ok,"(Instance_1, id_algorithm_0)"
7,Instance_1,id_algorithm_1,1.0,600.402,ok,"(Instance_1, id_algorithm_1)"
8,Instance_2,algorithm_0,1.0,3600.000,memout,"(Instance_2, algorithm_0)"
9,Instance_2,algorithm_1,1.0,3600.000,memout,"(Instance_2, algorithm_1)"


In [0]:
comp_keyed = alt.Chart(algReset).mark_circle(size=100).encode(
    x = 'runtime',
    y = 'runtime_y:Q',
    tooltip = ['instance_id', 
               'algorithm', 'runtime', 'runstatus',
               'algorithm_y:N', 'runtime_y:Q', 'runstatus_y:N'
              ]
).transform_lookup(
    lookup = 'key_col',
    from_ = alt.LookupData(algReset, key = 'key_col',
                           fields = ['runtime', 'algorithm', 'runstatus']),
    as_ = ['runtime_y', 'algorithm_y', 'runstatus_y']
)

comk_inst = comp_keyed.encode(color = 'instance_id')
comk_alg = comp_keyed.encode(color = 'algorithm')
comk_alg_y = comp_keyed.encode(color = 'algorithm_y:N')
comk_time = comp_keyed.encode(color = 'runtime')
comk_time_y = comp_keyed.encode(color = 'runtime_y:Q')
comk_stat = comp_keyed.encode(color = 'runstatus')
comk_stat_y = comp_keyed.encode(color = 'runstatus_y:N')

comk_inst & (comk_alg | comk_alg_y) & (comk_time | comk_time_y) & (comk_stat | comk_stat_y)

VConcatChart({
  data:    instance_id       algorithm  ...  runstatus                       key_col
  0   Instance_0     algorithm_0  ...     memout     (Instance_0, algorithm_0)
  1   Instance_0     algorithm_1  ...     memout     (Instance_0, algorithm_1)
  2   Instance_0  id_algorithm_0  ...         ok  (Instance_0, id_algorithm_0)
  3   Instance_0  id_algorithm_1  ...    timeout  (Instance_0, id_algorithm_1)
  4   Instance_1     algorithm_0  ...     memout     (Instance_1, algorithm_0)
  5   Instance_1     algorithm_1  ...     memout     (Instance_1, algorithm_1)
  6   Instance_1  id_algorithm_0  ...         ok  (Instance_1, id_algorithm_0)
  7   Instance_1  id_algorithm_1  ...         ok  (Instance_1, id_algorithm_1)
  8   Instance_2     algorithm_0  ...     memout     (Instance_2, algorithm_0)
  9   Instance_2     algorithm_1  ...     memout     (Instance_2, algorithm_1)
  10  Instance_2  id_algorithm_0  ...         ok  (Instance_2, id_algorithm_0)
  11  Instance_2  id_algorithm_1  ...         ok  (Instance_2, id_algorithm_1)
  12  Instance_3     algorithm_0  ...     memout     (Instance_3, algorithm_0)
  13  Instance_3     algorithm_1  ...         ok     (Instance_3, algorithm_1)
  14  Instance_3  id_algorithm_0  ...         ok  (Instance_3, id_algorithm_0)
  15  Instance_3  id_algorithm_1  ...         ok  (Instance_3, id_algorithm_1)
  16  Instance_4     algorithm_0  ...     memout     (Instance_4, algorithm_0)
  17  Instance_4     algorithm_1  ...     memout     (Instance_4, algorithm_1)
  18  Instance_4  id_algorithm_0  ...         ok  (Instance_4, id_algorithm_0)
  19  Instance_4  id_algorithm_1  ...    timeout  (Instance_4, id_algorithm_1)
  20  Instance_5     algorithm_0  ...         ok     (Instance_5, algorithm_0)
  21  Instance_5     algorithm_1  ...     memout     (Instance_5, algorithm_1)
  22  Instance_5  id_algorithm_0  ...         ok  (Instance_5, id_algorithm_0)
  23  Instance_5  id_algorithm_1  ...         ok  (Instance_5, id_algorithm_1)
  
  [24 rows x 6 columns],
  vconcat: [Chart({
    encoding: FacetedEncoding({
      color: Color({
        field: 'instance_id',
        type: 'nominal'
      }),
      tooltip: [Tooltip({
        field: 'instance_id',
        type: 'nominal'
      }), Tooltip({
        field: 'algorithm',
        type: 'nominal'
      }), Tooltip({
        field: 'runtime',
        type: 'quantitative'
      }), Tooltip({
        field: 'runstatus',
        type: 'nominal'
      }), Tooltip({
        field: 'algorithm_y',
        type: 'nominal'
      }), Tooltip({
        field: 'runtime_y',
        type: 'quantitative'
      }), Tooltip({
        field: 'runstatus_y',
        type: 'nominal'
      })],
      x: X({
        field: 'runtime',
        type: 'quantitative'
      }),
      y: Y({
        field: 'runtime_y',
        type: 'quantitative'
      })
    }),
    mark: MarkDef({
      size: 100,
      type: 'circle'
    }),
    transform: [LookupTransform({
      as: ['runtime_y', 'algorithm_y', 'runstatus_y'],
      from: LookupData({
        data:    instance_id       algorithm  ...  runstatus                       key_col
        0   Instance_0     algorithm_0  ...     memout     (Instance_0, algorithm_0)
        1   Instance_0     algorithm_1  ...     memout     (Instance_0, algorithm_1)
        2   Instance_0  id_algorithm_0  ...         ok  (Instance_0, id_algorithm_0)
        3   Instance_0  id_algorithm_1  ...    timeout  (Instance_0, id_algorithm_1)
        4   Instance_1     algorithm_0  ...     memout     (Instance_1, algorithm_0)
        5   Instance_1     algorithm_1  ...     memout     (Instance_1, algorithm_1)
        6   Instance_1  id_algorithm_0  ...         ok  (Instance_1, id_algorithm_0)
        7   Instance_1  id_algorithm_1  ...         ok  (Instance_1, id_algorithm_1)
        8   Instance_2     algorithm_0  ...     memout     (Instance_2, algorithm_0)
        9   Instance_2     algorithm_1  ...     memout     (Instance_2, algorithm_1)
        10  Instan

In [0]:
algSource.head()

,instance_id,repetition,algorithm,runtime,runstatus
0,Instance_0,1.0,algorithm_0,3600.000,memout
1,Instance_0,1.0,algorithm_1,3600.000,memout
2,Instance_0,1.0,id_algorithm_0,311.863,ok
3,Instance_0,1.0,id_algorithm_1,3600.000,timeout
4,Instance_1,1.0,algorithm_0,3600.000,memout


In [0]:
algKeyed = algSource.reset_index()
algKeyed.head()

,index,instance_id,repetition,algorithm,runtime,runstatus
0,0,Instance_0,1.0,algorithm_0,3600.000,memout
1,1,Instance_0,1.0,algorithm_1,3600.000,memout
2,2,Instance_0,1.0,id_algorithm_0,311.863,ok
3,3,Instance_0,1.0,id_algorithm_1,3600.000,timeout
4,4,Instance_1,1.0,algorithm_0,3600.000,memout


In [0]:
alt.Chart(algKeyed).mark_circle(size=100).encode(
    x = 'runtime',
    y = 'runtime_y:Q'
).transform_lookup(
    lookup = 'instance_id',
    from_ = alt.LookupData(algKeyed, key = 'index',
                           fields = ['runtime']
                          ),
    as_ = ['runtime_y']
)

Chart({
  data:     index instance_id  repetition       algorithm   runtime runstatus
  0       0  Instance_0         1.0     algorithm_0  3600.000    memout
  1       1  Instance_0         1.0     algorithm_1  3600.000    memout
  2       2  Instance_0         1.0  id_algorithm_0   311.863        ok
  3       3  Instance_0         1.0  id_algorithm_1  3600.000   timeout
  4       4  Instance_1         1.0     algorithm_0  3600.000    memout
  5       5  Instance_1         1.0     algorithm_1  3600.000    memout
  6       6  Instance_1         1.0  id_algorithm_0    78.121        ok
  7       7  Instance_1         1.0  id_algorithm_1   600.402        ok
  8       8  Instance_2         1.0     algorithm_0  3600.000    memout
  9       9  Instance_2         1.0     algorithm_1  3600.000    memout
  10     10  Instance_2         1.0  id_algorithm_0     0.012        ok
  11     11  Instance_2         1.0  id_algorithm_1    60.720        ok
  12     12  Instance_3         1.0     algorithm_0  3600.000    memout
  13     13  Instance_3         1.0     algorithm_1    92.986        ok
  14     14  Instance_3         1.0  id_algorithm_0  2031.039        ok
  15     15  Instance_3         1.0  id_algorithm_1   141.857        ok
  16     16  Instance_4         1.0     algorithm_0  3600.000    memout
  17     17  Instance_4         1.0     algorithm_1  3600.000    memout
  18     18  Instance_4         1.0  id_algorithm_0   422.530        ok
  19     19  Instance_4         1.0  id_algorithm_1  3600.000   timeout
  20     20  Instance_5         1.0     algorithm_0   182.019        ok
  21     21  Instance_5         1.0     algorithm_1  3600.000    memout
  22     22  Instance_5         1.0  id_algorithm_0    66.988        ok
  23     23  Instance_5         1.0  id_algorithm_1   249.488        ok,
  encoding: FacetedEncoding({
    x: X({
      field: 'runtime',
      type: 'quantitative'
    }),
    y: Y({
      field: 'runtime_y',
      type: 'quantitative'
    })
  }),
  mark: MarkDef({
    size: 100,
    type: 'circle'
  }),
  transform: [LookupTransform({
    as: ['runtime_y'],
    from: LookupData({
      data:     index instance_id  repetition       algorithm   runtime runstatus
      0       0  Instance_0         1.0     algorithm_0  3600.000    memout
      1       1  Instance_0         1.0     algorithm_1  3600.000    memout
      2       2  Instance_0         1.0  id_algorithm_0   311.863        ok
      3       3  Instance_0         1.0  id_algorithm_1  3600.000   timeout
      4       4  Instance_1         1.0     algorithm_0  3600.000    memout
      5       5  Instance_1         1.0     algorithm_1  3600.000    memout
      6       6  Instance_1         1.0  id_algorithm_0    78.121        ok
      7       7  Instance_1         1.0  id_algorithm_1   600.402        ok
      8       8  Instance_2         1.0     algorithm_0  3600.000    memout
      9       9  Instance_2         1.0     algorithm_1  3600.000    memout
      10     10  Instance_2         1.0  id_algorithm_0     0.012        ok
      11     11  Instance_2         1.0  id_algorithm_1    60.720        ok
      12     12  Instance_3         1.0     algorithm_0  3600.000    memout
      13     13  Instance_3         1.0     algorithm_1    92.986        ok
      14     14  Instance_3         1.0  id_algorithm_0  2031.039        ok
      15     15  Instance_3         1.0  id_algorithm_1   141.857        ok
      16     16  Instance_4         1.0     algorithm_0  3600.000    memout
      17     17  Instance_4         1.0     algorithm_1  3600.000    memout
      18     18  Instance_4         1.0  id_algorithm_0   422.530        ok
      19     19  Instance_4         1.0  id_algorithm_1  3600.000   timeout
      20     20  Instance_5         1.0     algorithm_0   182.019        ok
      21     21  Instance_5         1.0     algorithm_1  3600.000    memout
      22     22  Instance_5         1.0  id_algorithm_0    66.988        ok
      23     23  Instance_5         1.0  i

In [0]:
algList = list(algSource['algorithm'].unique())
algList

['algorithm_0', 'algorithm_1', 'id_algorithm_0', 'id_algorithm_1']

add a column for each possible combo, then perform a lookup on each. this would repeat processing steps but keep the data source the same length.

since my test source only has four algorithms, i'll start by doing this hard-coded, and figure out how to abstract it once i understand the structure.

except, multiple lookups can't go in the same column, or they overwrite previous lookups. and we're right back to the "can't dynamically change which column goes to which axis" problem

In [0]:
a0_col = pd.Series(map(lambda x : (x, 'algorithm_0'), algSource['instance_id']))
a1_col = pd.Series(map(lambda x : (x, 'algorithm_1'), algSource['instance_id']))
id_a0_col = pd.Series(map(lambda x : (x, 'id_algorithm_0'), algSource['instance_id']))
id_a1_col = pd.Series(map(lambda x : (x, 'id_algorithm_1'), algSource['instance_id']))

algTupled = algReset

algTupled['a0_col'] = a0_col
algTupled['a1_col'] = a1_col
algTupled['id_a0_col'] = id_a0_col
algTupled['id_a1_col'] = id_a1_col

#algTupled.head()

alt.Chart(algTupled).mark_circle(size=100).encode(
    x = 'runtime',
    y = 'runtime_y:Q',
    color = alt.Color('algorithm_y:N'),
    tooltip = ['instance_id', 'algorithm', 'runtime', 'algorithm_y:N', 'runtime_y:Q']
).transform_lookup(
    lookup = 'a0_col',
    from_ = alt.LookupData(algTupled, key = 'key_col',
                           fields = ['algorithm', 'runtime']),
    as_ = ['algorithm_y', 'runtime_y']    
).transform_lookup(
    lookup = 'a1_col',
    from_ = alt.LookupData(algTupled, key = 'key_col',
                            fields = ['algorithm', 'runtime']),
    as_ = ['algorithm_y', 'runtime_y']
)

Chart({
  data:    instance_id  ...                     id_a1_col
  0   Instance_0  ...  (Instance_0, id_algorithm_1)
  1   Instance_0  ...  (Instance_0, id_algorithm_1)
  2   Instance_0  ...  (Instance_0, id_algorithm_1)
  3   Instance_0  ...  (Instance_0, id_algorithm_1)
  4   Instance_1  ...  (Instance_1, id_algorithm_1)
  5   Instance_1  ...  (Instance_1, id_algorithm_1)
  6   Instance_1  ...  (Instance_1, id_algorithm_1)
  7   Instance_1  ...  (Instance_1, id_algorithm_1)
  8   Instance_2  ...  (Instance_2, id_algorithm_1)
  9   Instance_2  ...  (Instance_2, id_algorithm_1)
  10  Instance_2  ...  (Instance_2, id_algorithm_1)
  11  Instance_2  ...  (Instance_2, id_algorithm_1)
  12  Instance_3  ...  (Instance_3, id_algorithm_1)
  13  Instance_3  ...  (Instance_3, id_algorithm_1)
  14  Instance_3  ...  (Instance_3, id_algorithm_1)
  15  Instance_3  ...  (Instance_3, id_algorithm_1)
  16  Instance_4  ...  (Instance_4, id_algorithm_1)
  17  Instance_4  ...  (Instance_4, id_algorithm_1)
  18  Instance_4  ...  (Instance_4, id_algorithm_1)
  19  Instance_4  ...  (Instance_4, id_algorithm_1)
  20  Instance_5  ...  (Instance_5, id_algorithm_1)
  21  Instance_5  ...  (Instance_5, id_algorithm_1)
  22  Instance_5  ...  (Instance_5, id_algorithm_1)
  23  Instance_5  ...  (Instance_5, id_algorithm_1)
  
  [24 rows x 10 columns],
  encoding: FacetedEncoding({
    color: Color({
      field: 'algorithm_y',
      type: 'nominal'
    }),
    tooltip: [Tooltip({
      field: 'instance_id',
      type: 'nominal'
    }), Tooltip({
      field: 'algorithm',
      type: 'nominal'
    }), Tooltip({
      field: 'runtime',
      type: 'quantitative'
    }), Tooltip({
      field: 'algorithm_y',
      type: 'nominal'
    }), Tooltip({
      field: 'runtime_y',
      type: 'quantitative'
    })],
    x: X({
      field: 'runtime',
      type: 'quantitative'
    }),
    y: Y({
      field: 'runtime_y',
      type: 'quantitative'
    })
  }),
  mark: MarkDef({
    size: 100,
    type: 'circle'
  }),
  transform: [LookupTransform({
    as: ['algorithm_y', 'runtime_y'],
    from: LookupData({
      data:    instance_id  ...                     id_a1_col
      0   Instance_0  ...  (Instance_0, id_algorithm_1)
      1   Instance_0  ...  (Instance_0, id_algorithm_1)
      2   Instance_0  ...  (Instance_0, id_algorithm_1)
      3   Instance_0  ...  (Instance_0, id_algorithm_1)
      4   Instance_1  ...  (Instance_1, id_algorithm_1)
      5   Instance_1  ...  (Instance_1, id_algorithm_1)
      6   Instance_1  ...  (Instance_1, id_algorithm_1)
      7   Instance_1  ...  (Instance_1, id_algorithm_1)
      8   Instance_2  ...  (Instance_2, id_algorithm_1)
      9   Instance_2  ...  (Instance_2, id_algorithm_1)
      10  Instance_2  ...  (Instance_2, id_algorithm_1)
      11  Instance_2  ...  (Instance_2, id_algorithm_1)
      12  Instance_3  ...  (Instance_3, id_algorithm_1)
      13  Instance_3  ...  (Instance_3, id_algorithm_1)
      14  Instance_3  ...  (Instance_3, id_algorithm_1)
      15  Instance_3  ...  (Instance_3, id_algorithm_1)
      16  Instance_4  ...  (Instance_4, id_algorithm_1)
      17  Instance_4  ...  (Instance_4, id_algorithm_1)
      18  Instance_4  ...  (Instance_4, id_algorithm_1)
      19  Instance_4  ...  (Instance_4, id_algorithm_1)
      20  Instance_5  ...  (Instance_5, id_algorithm_1)
      21  Instance_5  ...  (Instance_5, id_algorithm_1)
      22  Instance_5  ...  (Instance_5, id_algorithm_1)
      23  Instance_5  ...  (Instance_5, id_algorithm_1)
      
      [24 rows x 10 columns],
      fields: ['algorithm', 'runtime'],
      key: 'key_col'
    }),
    lookup: 'a0_col'
  }), LookupTransform({
    as: ['algorithm_y', 'runtime_y'],
    from: LookupData({
      data:    instance_id  ...                     id_a1_col
      0   Instance_0  ...  (Instance_0, id_algorithm_1)
      1   Instance_0  ...  (Instance_0, id_algorithm_1)
      2   Instance_0  ...  (Instance_0, id_algorithm_1)
      3   Instance_0  ...  (Instance_0, id_algo

i probably need to revisit plotting large data sets, so i can use my joined one!!

what about y2? y3?

...y3 doesn't exist, and y2 isn't plotted in scatter

In [0]:
corgis.head()

,Name,Source,Is this Real or Not Real?,Cuteness,Arbitrary numbers 1-5
0,Holly,Queen of England,Real,7,1
1,Ralph,Instagram,Real,7,2
2,"Molly (""Thing of Evil"")",Stephen King,Real,9,3
3,Ein,Cowboy Bebop,Not Real,11,4
4,Loki,Instagram,Real,8,5


In [0]:
alt.Chart(corgis).mark_circle(size=100).encode(
    x = 'Name',
    y = 'Cuteness',
    y2 = 'Arbitrary numbers 1-5'
)

Chart({
  data:                       Name  ... Arbitrary numbers 1-5
  0                    Holly  ...                     1
  1                    Ralph  ...                     2
  2  Molly ("Thing of Evil")  ...                     3
  3                      Ein  ...                     4
  4                     Loki  ...                     5
  5                    Rufus  ...                     1
  6             Sutter Brown  ...                     2
  7               Hope Corgi  ...                     3
  
  [8 rows x 5 columns],
  encoding: FacetedEncoding({
    x: X({
      field: 'Name',
      type: 'nominal'
    }),
    y: Y({
      field: 'Cuteness',
      type: 'quantitative'
    }),
    y2: Y2({
      field: 'Arbitrary numbers 1-5'
    })
  }),
  mark: MarkDef({
    size: 100,
    type: 'circle'
  })
})

In [0]:
alt.Chart(corgis).mark_area().encode(
    x = 'Name',
    y = 'Cuteness',
    y2 = 'Arbitrary numbers 1-5'
)

Chart({
  data:                       Name  ... Arbitrary numbers 1-5
  0                    Holly  ...                     1
  1                    Ralph  ...                     2
  2  Molly ("Thing of Evil")  ...                     3
  3                      Ein  ...                     4
  4                     Loki  ...                     5
  5                    Rufus  ...                     1
  6             Sutter Brown  ...                     2
  7               Hope Corgi  ...                     3
  
  [8 rows x 5 columns],
  encoding: FacetedEncoding({
    x: X({
      field: 'Name',
      type: 'nominal'
    }),
    y: Y({
      field: 'Cuteness',
      type: 'quantitative'
    }),
    y2: Y2({
      field: 'Arbitrary numbers 1-5'
    })
  }),
  mark: 'area'
})

I asked the devs:
https://github.com/altair-viz/altair/issues/1617

Apparently, selectors can be created with an initial value, as well!

In [1]:
import altair as alt
from vega_datasets import data


columns = ['Acceleration', 'Cylinders', 'Displacement', 'Horsepower']

select_box = alt.binding_select(options=columns, name='column')
sel = alt.selection_single(fields=['column'], bind=select_box, init={'column': 'Acceleration'})

alt.Chart(data.cars()).transform_fold(
    columns,
    as_=['column', 'value']
).transform_filter(
    sel  
).mark_point().encode(
    x='Miles_per_Gallon:Q',
    y='value:Q',
    color='Origin:N'
).add_selection(
    sel
)

Chart({
  data:      Acceleration  Cylinders  Displacement  ...  Origin  Weight_in_lbs       Year
  0            12.0          8         307.0  ...     USA           3504 1970-01-01
  1            11.5          8         350.0  ...     USA           3693 1970-01-01
  2            11.0          8         318.0  ...     USA           3436 1970-01-01
  3            12.0          8         304.0  ...     USA           3433 1970-01-01
  4            10.5          8         302.0  ...     USA           3449 1970-01-01
  5            10.0          8         429.0  ...     USA           4341 1970-01-01
  6             9.0          8         454.0  ...     USA           4354 1970-01-01
  7             8.5          8         440.0  ...     USA           4312 1970-01-01
  8            10.0          8         455.0  ...     USA           4425 1970-01-01
  9             8.5          8         390.0  ...     USA           3850 1970-01-01
  10           17.5          4         133.0  ...  Europe           3090 1970-01-01
  11           11.5          8         350.0  ...     USA           4142 1970-01-01
  12           11.0          8         351.0  ...     USA           4034 1970-01-01
  13           10.5          8         383.0  ...     USA           4166 1970-01-01
  14           11.0          8         360.0  ...     USA           3850 1970-01-01
  15           10.0          8         383.0  ...     USA           3563 1970-01-01
  16            8.0          8         340.0  ...     USA           3609 1970-01-01
  17            8.0          8         302.0  ...     USA           3353 1970-01-01
  18            9.5          8         400.0  ...     USA           3761 1970-01-01
  19           10.0          8         455.0  ...     USA           3086 1970-01-01
  20           15.0          4         113.0  ...   Japan           2372 1970-01-01
  21           15.5          6         198.0  ...     USA           2833 1970-01-01
  22           15.5          6         199.0  ...     USA           2774 1970-01-01
  23           16.0          6         200.0  ...     USA           2587 1970-01-01
  24           14.5          4          97.0  ...   Japan           2130 1970-01-01
  25           20.5          4          97.0  ...  Europe           1835 1970-01-01
  26           17.5          4         110.0  ...  Europe           2672 1970-01-01
  27           14.5          4         107.0  ...  Europe           2430 1970-01-01
  28           17.5          4         104.0  ...  Europe           2375 1970-01-01
  29           12.5          4         121.0  ...  Europe           2234 1970-01-01
  ..            ...        ...           ...  ...     ...            ...        ...
  376          18.6          4         112.0  ...     USA           2640 1982-01-01
  377          18.0          4         112.0  ...     USA           2395 1982-01-01
  378          16.2          4         112.0  ...     USA           2575 1982-01-01
  379          16.0          4         135.0  ...     USA           2525 1982-01-01
  380          18.0          4         151.0  ...     USA           2735 1982-01-01
  381          16.4          4         140.0  ...     USA           2865 1982-01-01
  382          20.5          4         151.0  ...     USA           3035 1982-01-01
  383          15.3          4         105.0  ...  Europe           1980 1982-01-01
  384          18.2          4          91.0  ...   Japan           2025 1982-01-01
  385          17.6          4          91.0  ...   Japan           1970 1982-01-01
  386          14.7          4         105.0  ...     USA           2125 1982-01-01
  387          17.3          4          98.0  ...     USA           2125 1982-01-01
  388          14.5          4         120.0  ...   Japan           2160 1982-01-01
  389          14.5          4         107.0  ...   Japan           2205 1982-01-01
  390          16.9          4         108.0  ...   Japan           2245 1982-01-01
  391          15.0          4        

In [10]:
algMulti.head()

repetition   runtime runstatus
instance_id algorithm                                     
Instance_0  algorithm_0            1.0  3600.000    memout
            algorithm_1            1.0  3600.000    memout
            id_algorithm_0         1.0   311.863        ok
            id_algorithm_1         1.0  3600.000   timeout
Instance_1  algorithm_0            1.0  3600.000    memout

In [22]:
algFlat.head()

,repetition_algorithm_0,repetition_algorithm_1,repetition_id_algorithm_0,repetition_id_algorithm_1,runtime_algorithm_0,runtime_algorithm_1,runtime_id_algorithm_0,runtime_id_algorithm_1,runstatus_algorithm_0,runstatus_algorithm_1,runstatus_id_algorithm_0,runstatus_id_algorithm_1
instance_id,,,,,,,,,,,,
Instance_0,1.0,1.0,1.0,1.0,3600.0,3600.000,311.863,3600.000,memout,memout,ok,timeout
Instance_1,1.0,1.0,1.0,1.0,3600.0,3600.000,78.121,600.402,memout,memout,ok,ok
Instance_2,1.0,1.0,1.0,1.0,3600.0,3600.000,0.012,60.720,memout,memout,ok,ok
Instance_3,1.0,1.0,1.0,1.0,3600.0,92.986,2031.039,141.857,memout,ok,ok,ok
Instance_4,1.0,1.0,1.0,1.0,3600.0,3600.000,422.530,3600.000,memout,memout,ok,timeout


In [24]:
runtimes = ['runtime_algorithm_1', 
            'runtime_id_algorithm_0', 'runtime_id_algorithm_1']

alt.Chart(algFlat).transform_fold(
    runtimes,
    as_ = ['algorithm', 'runtime']
).mark_circle().encode(
    x = 'runtime_algorithm_0',
    y = 'runtime:Q',
    color = 'algorithm:N'
)

Chart({
  data:              repetition_algorithm_0  ...  runstatus_id_algorithm_1
  instance_id                          ...                          
  Instance_0                      1.0  ...                   timeout
  Instance_1                      1.0  ...                        ok
  Instance_2                      1.0  ...                        ok
  Instance_3                      1.0  ...                        ok
  Instance_4                      1.0  ...                   timeout
  Instance_5                      1.0  ...                        ok
  
  [6 rows x 12 columns],
  encoding: FacetedEncoding({
    color: Color({
      field: 'algorithm',
      type: 'nominal'
    }),
    x: X({
      field: 'runtime_algorithm_0',
      type: 'quantitative'
    }),
    y: Y({
      field: 'runtime',
      type: 'quantitative'
    })
  }),
  mark: 'circle',
  transform: [FoldTransform({
    as: ['algorithm', 'runtime'],
    fold: ['runtime_algorithm_1', 'runtime_id_algorithm_0', 'runtime_id_algorithm_1']
  })]
})

In [30]:
runtimes = ['runtime_algorithm_0', 'runtime_algorithm_1', 
            'runtime_id_algorithm_0', 'runtime_id_algorithm_1']

chart = alt.Chart(algFlat).transform_fold(
    runtimes,
    as_ = ['algorithm_x', 'runtime_x']
).transform_fold(
    runtimes,
    as_ = ['algorithm_y', 'runtime_y']
).mark_circle().encode(
    x = 'runtime_x:Q',
    y = 'runtime_y:Q'
)

pprint(chart.to_dict())

{'$schema': 'https://vega.github.io/schema/vega-lite/v3.3.0.json',
 'config': {'mark': {'tooltip': None}, 'view': {'height': 300, 'width': 400}},
 'data': {'name': 'data-4dbaa2bcb0eb648a132ae9c53ba2510b'},
 'datasets': {'data-4dbaa2bcb0eb648a132ae9c53ba2510b': [{'repetition_algorithm_0': 1.0,
                                                         'repetition_algorithm_1': 1.0,
                                                         'repetition_id_algorithm_0': 1.0,
                                                         'repetition_id_algorithm_1': 1.0,
                                                         'runstatus_algorithm_0': 'memout',
                                                         'runstatus_algorithm_1': 'memout',
                                                         'runstatus_id_algorithm_0': 'ok',
                                                         'runstatus_id_algorithm_1': 'timeout',
                                                         'runtime

In [31]:
chart

Chart({
  data:              repetition_algorithm_0  ...  runstatus_id_algorithm_1
  instance_id                          ...                          
  Instance_0                      1.0  ...                   timeout
  Instance_1                      1.0  ...                        ok
  Instance_2                      1.0  ...                        ok
  Instance_3                      1.0  ...                        ok
  Instance_4                      1.0  ...                   timeout
  Instance_5                      1.0  ...                        ok
  
  [6 rows x 12 columns],
  encoding: FacetedEncoding({
    x: X({
      field: 'runtime_x',
      type: 'quantitative'
    }),
    y: Y({
      field: 'runtime_y',
      type: 'quantitative'
    })
  }),
  mark: 'circle',
  transform: [FoldTransform({
    as: ['algorithm_x', 'runtime_x'],
    fold: ['runtime_algorithm_0', 'runtime_algorithm_1', 'runtime_id_algorithm_0', 'runtime_id_algorithm_1']
  }), FoldTransform({
    as: ['algorithm_y', 'runtime_y'],
    fold: ['runtime_algorithm_0', 'runtime_algorithm_1', 'runtime_id_algorithm_0', 'runtime_id_algorithm_1']
  })]
})